In [1]:
from pyspark.sql.functions import *
from pyspark.sql import Window


StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 3, Finished, Available, Finished)

# Loading all file in notebook as DFs

In [2]:
df_bo = spark.read.format("csv").load("Files/BoxOffice Project/Datasets_Final/Boxoffice_Fact.csv",header=True,inferSchema=True)
df_dir = spark.read.format("csv").load("Files/BoxOffice Project/Datasets_Final/Director_dim.csv",header = True,inferSchema =True)
df_gen = spark.read.format("csv").load("Files/BoxOffice Project/Datasets_Final/Genere_dim.csv",header=True,inferSchema=True)
df_lan = spark.read.format("csv").load("Files/BoxOffice Project/Datasets_Final/Language_dim.csv",header = True,inferSchema=True)

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 4, Finished, Available, Finished)

# To check the schema of dataframe

In [3]:
df_bo.printSchema()
df_dir.printSchema()
df_gen.printSchema()
df_lan.printSchema()

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 5, Finished, Available, Finished)

root
 |-- FilmID: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- DirectorID: integer (nullable = true)
 |-- Lead_Actor/Actress: string (nullable = true)
 |-- LanguageID: integer (nullable = true)
 |-- Industry: string (nullable = true)
 |-- GenreID: integer (nullable = true)
 |-- Budget_in_Crores: string (nullable = true)
 |-- First_Day_Collection_Worldwide_in_Crores: string (nullable = true)
 |-- Worldwide_Collection _in_Crores: string (nullable = true)
 |-- Overseas_Collection_in_Crores: string (nullable = true)
 |-- India_Gross_Collection_in_Crores: string (nullable = true)
 |-- Verdict: string (nullable = true)
 |-- IMDb_Rating: double (nullable = true)
 |-- Runtime_(mins): integer (nullable = true)
 |-- OTT_Platform: string (nullable = true)

root
 |-- Director: string (nullable = true)
 |-- Director_ID: integer (nullable = true)

root
 |-- Genre: string (nullable = true)
 |-- GenreID: integer (nullable = true)

root

# to get all the columns missing values

In [4]:
def check_missvalues(df,lstcl):
    missing_values = {}
    for i in lstcl:
        a = df.filter(col(i).isNull()).count()
        missing_values[i] = a
    return ("missing_values",missing_values)

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 6, Finished, Available, Finished)

In [5]:
check_missvalues(df_bo,df_bo.columns)

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 7, Finished, Available, Finished)

('missing_values',
 {'FilmID': 0,
  'Title': 0,
  'Release_Date': 0,
  'DirectorID': 0,
  'Lead_Actor/Actress': 0,
  'LanguageID': 0,
  'Industry': 0,
  'GenreID': 0,
  'Budget_in_Crores': 0,
  'First_Day_Collection_Worldwide_in_Crores': 0,
  'Worldwide_Collection _in_Crores': 0,
  'Overseas_Collection_in_Crores': 9,
  'India_Gross_Collection_in_Crores': 0,
  'Verdict': 0,
  'IMDb_Rating': 0,
  'Runtime_(mins)': 0,
  'OTT_Platform': 0})

In [6]:
check_missvalues(df_dir,df_dir.columns)

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 8, Finished, Available, Finished)

('missing_values', {'Director': 0, 'Director_ID': 0})

In [7]:
check_missvalues(df_gen,df_gen.columns)

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 9, Finished, Available, Finished)

('missing_values', {'Genre': 0, 'GenreID': 0})

In [8]:
check_missvalues(df_lan,df_lan.columns)

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 10, Finished, Available, Finished)

('missing_values', {'Language': 0, 'LanguageID': 0})

In [9]:
display(df_bo.head(5))

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 39b4e0d2-2bb4-436b-b274-6cc5a51ee018)

In [10]:
df_bo.select("LanguageID","Industry").distinct().show()

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 12, Finished, Available, Finished)

+----------+----------+
|LanguageID|  Industry|
+----------+----------+
|       513| Mollywood|
|       511| Tollywood|
|       519|Sandalwood|
|       503| Kollywood|
|       507|Sandalwood|
|       501| Bollywood|
|       505| Tollywood|
|       509| Mollywood|
|       515| Bollywood|
|       517| Kollywood|
+----------+----------+



In [11]:
df_lan.show()

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 13, Finished, Available, Finished)

+---------+----------+
| Language|LanguageID|
+---------+----------+
|    Hindi|       501|
|    Tamil|       503|
|   Telugu|       505|
|  Kannada|       507|
|Malayalam|       509|
+---------+----------+



In [12]:
# Hindi --> Bollywood --> 515 with 501
# Telugu --> Tollywood --> 511 with 505
# Tamil --> Kollywood --> 517 with 503
# Kannada --> Sandalwood --> 519 with 507
# Malayalam --> Mollywood --> 513 with 509
# now we need to replace the above on LanguageID column in df_bo

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 14, Finished, Available, Finished)

In [13]:
df_bo = df_bo.withColumn("LanguageID", when(col("LanguageID") == 515,501)
                                       .when(col("LanguageID") == 511,505)
                                       .when(col("LanguageID") == 517,503)
                                       .when(col("LanguageID") == 519,507)
                                       .when(col("LanguageID")== 513,509)
                                       .otherwise(col("LanguageID")))

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 15, Finished, Available, Finished)

In [14]:
df_bo.select("LanguageID","Industry").distinct().show()

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 16, Finished, Available, Finished)

+----------+----------+
|LanguageID|  Industry|
+----------+----------+
|       503| Kollywood|
|       507|Sandalwood|
|       501| Bollywood|
|       505| Tollywood|
|       509| Mollywood|
+----------+----------+



# Numerical columns special characters & white spaces identification

In [15]:
# def detect_special_char(df):
#     for column in df.columns:
#         df = df.withColumn(f"{column}_Hashwhitespace", col(column).rlike(r"\s"))
#         df = df.withColumn(f"{column}_Hasspecialchar", col(column).rlike(r"[^a-zA-Z0-9\s]"))
#     return display(df)

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 17, Finished, Available, Finished)

In [16]:
#define regex patterens
whitespace_patteren = r"\s+"
Spec_char_patteren = r"[^a-zA-Z0-9]"

def count_spec_chars(df, column):
    return df.withColumn("WhiteSpace", regexp_extract(col(column),whitespace_patteren, 0))\
             .withColumn("Special_Characters",regexp_extract(col(column),Spec_char_patteren,0))\
             .groupBy(column,"WhiteSpace","Special_Characters").agg(count("*").alias("Count"))

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 18, Finished, Available, Finished)

In [17]:
count_spec_chars(df_bo,"Budget_in_Crores" ).show(truncate=False)

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 19, Finished, Available, Finished)

+----------------+----------+------------------+-----+
|Budget_in_Crores|WhiteSpace|Special_Characters|Count|
+----------------+----------+------------------+-----+
|540             |          |                  |1    |
|: 10            |          |:                 |2    |
|: 7             |          |:                 |1    |
|8               |          |                  |11   |
|28              |          |                  |1    |
|4               |          |                  |8    |
|2               |          |                  |2    |
|: 60            |          |:                 |1    |
|18              |          |                  |4    |
|: 6             |          |:                 |1    |
|8.5             |          |.                 |1    |
|3               |          |                  |4    |
|65              |          |                  |17   |
|: 100           |          |:                 |1    |
|175             |          |                  |1    |
|: 4      

In [18]:
count_spec_chars(df_bo,"First_Day_Collection_Worldwide_in_Crores").show()

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 20, Finished, Available, Finished)

+----------------------------------------+----------+------------------+-----+
|First_Day_Collection_Worldwide_in_Crores|WhiteSpace|Special_Characters|Count|
+----------------------------------------+----------+------------------+-----+
|                                     7.2|          |                 .|    4|
|                                     8.4|          |                 .|    3|
|                                     5.2|          |                 .|    1|
|                                    27.2|          |                 .|    1|
|                                    29.6|          |                 .|    1|
|                                     9.7|          |                 .|    2|
|                                    14.7|          |                 .|    1|
|                                     3.2|          |                 .|    2|
|                                       8|          |                  |    7|
|                                    18.3|          

In [19]:
count_spec_chars(df_bo,"Worldwide_Collection _in_Crores").show()

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 21, Finished, Available, Finished)

+-------------------------------+----------+------------------+-----+
|Worldwide_Collection _in_Crores|WhiteSpace|Special_Characters|Count|
+-------------------------------+----------+------------------+-----+
|                           33.9|          |                 .|    1|
|                          253.2|          |                 .|    1|
|                            137|          |                  |    1|
|                           45.4|          |                 .|    1|
|                             26|          |                  |    2|
|                           58.2|          |                 .|    1|
|                           28.2|          |                 .|    1|
|                          42.25|          |                 .|    1|
|                            218|          |                  |    1|
|                          236.4|          |                 .|    1|
|                          178.5|          |                 .|    1|
|                   

In [20]:
list(df_bo.columns)

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 22, Finished, Available, Finished)

['FilmID',
 'Title',
 'Release_Date',
 'DirectorID',
 'Lead_Actor/Actress',
 'LanguageID',
 'Industry',
 'GenreID',
 'Budget_in_Crores',
 'First_Day_Collection_Worldwide_in_Crores',
 'Worldwide_Collection _in_Crores',
 'Overseas_Collection_in_Crores',
 'India_Gross_Collection_in_Crores',
 'Verdict',
 'IMDb_Rating',
 'Runtime_(mins)',
 'OTT_Platform']

# Removing white spaces & : from the respective columns

In [21]:
df_bo = df_bo.withColumn("Budget_in_Crores", trim(regexp_replace(col("Budget_in_Crores"),"[:]", "")))

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 23, Finished, Available, Finished)

In [22]:
df_bo = df_bo.withColumn("First_Day_Collection_Worldwide_in_Crores"\
             ,trim(regexp_replace(col("First_Day_Collection_Worldwide_in_Crores"),"[:]", "")))
df_bo = df_bo.withColumn("Worldwide_Collection _in_Crores",trim(regexp_replace(col("Worldwide_Collection _in_Crores"),"[:]", "")))
df_bo = df_bo.withColumn("Overseas_Collection_in_Crores",trim(regexp_replace(col("Overseas_Collection_in_Crores"),"[:]", "")))
df_bo = df_bo.withColumn("India_Gross_Collection_in_Crores",trim(regexp_replace(col("India_Gross_Collection_in_Crores"),"[:]", "")))

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 24, Finished, Available, Finished)

In [23]:
count_spec_chars(df_bo,"Verdict").show() # we have white space and : in Verdict column. we need to fix this

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 25, Finished, Available, Finished)

+--------------------+----------+------------------+-----+
|             Verdict|WhiteSpace|Special_Characters|Count|
+--------------------+----------+------------------+-----+
|          : Disaster|          |                 :|    7|
|: All Time Blockb...|          |                 :|    2|
|     : Below Average|          |                 :|    4|
|             Average|          |                  |   35|
|     : Above Average|          |                 :|    2|
|All Time Blockbuster|          |                  |   14|
|         : Super Hit|          |                 :|    8|
|            Disaster|          |                  |   61|
|          Super Hitt|          |                  |    4|
|       Below Average|          |                  |    1|
|               : Hit|          |                 :|    9|
|       Below Average|          |                  |   42|
|       Above Average|          |                  |   21|
|                 Hit|          |                  |   8

In [24]:
df_bo = df_bo.withColumn("Verdict",regexp_replace(col("Verdict"),"[:]", "")) # Now replaced : with empty space

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 26, Finished, Available, Finished)

In [25]:
count_spec_chars(df_bo,"Verdict").show() # still there is a white space, we need to remove it.

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 27, Finished, Available, Finished)

+--------------------+----------+------------------+-----+
|             Verdict|WhiteSpace|Special_Characters|Count|
+--------------------+----------+------------------+-----+
|             Average|          |                  |   35|
|All Time Blockbuster|          |                  |   14|
|            Disaster|          |                  |   61|
|                 Hit|          |                  |    9|
|          Super Hitt|          |                  |    4|
|         Blockbuster|          |                  |   22|
|       Below Average|          |                  |    5|
|       Below Average|          |                  |   42|
|       Above Average|          |                  |   21|
|                 Hit|          |                  |   87|
|           Super Hit|          |                  |    1|
|           Super Hit|          |                  |   76|
|             Average|          |                  |    7|
|                Flop|          |                  |    

In [26]:
df_bo = df_bo.withColumn("Verdict",ltrim(col("Verdict"))) # replacing left side white space

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 28, Finished, Available, Finished)

In [27]:
count_spec_chars(df_bo,"Verdict").show() # still we have value called Super Hitt it's wrong need to replace with Super Hit

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 29, Finished, Available, Finished)

+--------------------+----------+------------------+-----+
|             Verdict|WhiteSpace|Special_Characters|Count|
+--------------------+----------+------------------+-----+
|             Average|          |                  |   42|
|All Time Blockbuster|          |                  |   16|
|            Disaster|          |                  |   68|
|          Super Hitt|          |                  |    4|
|       Below Average|          |                  |   47|
|       Above Average|          |                  |   23|
|                 Hit|          |                  |   96|
|           Super Hit|          |                  |   85|
|         Blockbuster|          |                  |  166|
|                Flop|          |                  |   57|
+--------------------+----------+------------------+-----+



In [28]:
df_bo = df_bo.withColumn("Verdict",when(col("Verdict") == "Super Hitt", "Super Hit").otherwise(col("Verdict")))

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 30, Finished, Available, Finished)

In [29]:
count_spec_chars(df_bo,"Verdict").show()

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 31, Finished, Available, Finished)

+--------------------+----------+------------------+-----+
|             Verdict|WhiteSpace|Special_Characters|Count|
+--------------------+----------+------------------+-----+
|             Average|          |                  |   42|
|All Time Blockbuster|          |                  |   16|
|            Disaster|          |                  |   68|
|       Below Average|          |                  |   47|
|       Above Average|          |                  |   23|
|                 Hit|          |                  |   96|
|           Super Hit|          |                  |   89|
|         Blockbuster|          |                  |  166|
|                Flop|          |                  |   57|
+--------------------+----------+------------------+-----+



In [30]:
df_bo.select("OTT_Platform").distinct().show() # There is white space in right side, need to trim that

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 32, Finished, Available, Finished)

+--------------------+
|        OTT_Platform|
+--------------------+
|                ZEE5|
|    Disney+ Hotstar |
|   Disney+ Hotstar  |
|               ZEE5 |
|             SonyLIV|
|          ZEE5      |
|  Google Play Movies|
|               Aha  |
|             Sun NXT|
|           Netflix  |
|            Netflix |
|           SonyLIV  |
| Amazon Prime Video |
|           Sun NXT  |
|               aha  |
|              ZEE5  |
|  Amazon Prime Video|
|     Disney+ Hotstar|
|              Voot  |
|Amazon Prime Video  |
+--------------------+
only showing top 20 rows



In [31]:
df_bo = df_bo.withColumn("OTT_Platform",rtrim(col("OTT_Platform"))) # Removing the white space at rightside

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 33, Finished, Available, Finished)

In [32]:
df_bo.select("OTT_Platform").distinct().show() #Now we have two aha's we need to replace aha with Aha

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 34, Finished, Available, Finished)

+------------------+
|      OTT_Platform|
+------------------+
|              ZEE5|
|               aha|
|           SonyLIV|
|              Voot|
|Google Play Movies|
|           Sun NXT|
|Amazon Prime Video|
|   Disney+ Hotstar|
|         JioCinema|
|               Aha|
|           Netflix|
+------------------+



In [33]:
df_bo = df_bo.withColumn("OTT_Platform", when(col("OTT_Platform") == "aha", "Aha").otherwise(col("OTT_Platform"))) #Replacing aha with Aha

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 35, Finished, Available, Finished)

In [34]:
df_bo.select("OTT_Platform").distinct().show()

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 36, Finished, Available, Finished)

+------------------+
|      OTT_Platform|
+------------------+
|              ZEE5|
|           SonyLIV|
|              Voot|
|Google Play Movies|
|           Sun NXT|
|Amazon Prime Video|
|   Disney+ Hotstar|
|         JioCinema|
|               Aha|
|           Netflix|
+------------------+



In [35]:
df_bo.printSchema()

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 37, Finished, Available, Finished)

root
 |-- FilmID: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- DirectorID: integer (nullable = true)
 |-- Lead_Actor/Actress: string (nullable = true)
 |-- LanguageID: integer (nullable = true)
 |-- Industry: string (nullable = true)
 |-- GenreID: integer (nullable = true)
 |-- Budget_in_Crores: string (nullable = true)
 |-- First_Day_Collection_Worldwide_in_Crores: string (nullable = true)
 |-- Worldwide_Collection _in_Crores: string (nullable = true)
 |-- Overseas_Collection_in_Crores: string (nullable = true)
 |-- India_Gross_Collection_in_Crores: string (nullable = true)
 |-- Verdict: string (nullable = true)
 |-- IMDb_Rating: double (nullable = true)
 |-- Runtime_(mins): integer (nullable = true)
 |-- OTT_Platform: string (nullable = true)



In [36]:
# Few columns data types are improper, so we need to convert them

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 38, Finished, Available, Finished)

In [37]:
df_bo = df_bo.withColumn("Budget_in_Crores",col("Budget_in_Crores").cast("double"))

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 39, Finished, Available, Finished)

In [38]:
df_bo = df_bo.withColumn("First_Day_Collection_Worldwide_in_Crores",col("First_Day_Collection_Worldwide_in_Crores").cast("double"))
df_bo = df_bo.withColumn("Overseas_Collection_in_Crores",col("Overseas_Collection_in_Crores").cast("double"))
df_bo = df_bo.withColumn("Worldwide_Collection _in_Crores",col("Worldwide_Collection _in_Crores").cast("double"))
df_bo = df_bo.withColumn("India_Gross_Collection_in_Crores",col("India_Gross_Collection_in_Crores").cast("double"))

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 40, Finished, Available, Finished)

In [39]:
df_bo = df_bo.withColumn("Release_Date",to_date(col("Release_Date"),"dd-MMM-yy"))

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 41, Finished, Available, Finished)

In [40]:
df_bo.printSchema()

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 42, Finished, Available, Finished)

root
 |-- FilmID: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Release_Date: date (nullable = true)
 |-- DirectorID: integer (nullable = true)
 |-- Lead_Actor/Actress: string (nullable = true)
 |-- LanguageID: integer (nullable = true)
 |-- Industry: string (nullable = true)
 |-- GenreID: integer (nullable = true)
 |-- Budget_in_Crores: double (nullable = true)
 |-- First_Day_Collection_Worldwide_in_Crores: double (nullable = true)
 |-- Worldwide_Collection _in_Crores: double (nullable = true)
 |-- Overseas_Collection_in_Crores: double (nullable = true)
 |-- India_Gross_Collection_in_Crores: double (nullable = true)
 |-- Verdict: string (nullable = true)
 |-- IMDb_Rating: double (nullable = true)
 |-- Runtime_(mins): integer (nullable = true)
 |-- OTT_Platform: string (nullable = true)



In [41]:
df_bo = df_bo.withColumnRenamed("Worldwide_Collection _in_Crores","Worldwide_Collection_in_Crores") 
# removing the white space and correcting with proper name

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 43, Finished, Available, Finished)

In [42]:
display(df_bo.head(5))

StatementMeta(, 7775a05d-0f2a-474b-8cda-bdaf5eb9e1b7, 44, Submitted, Running, Running)

In [ ]:
df_bo.printSchema()

StatementMeta(, , -1, Waiting, , Waiting)

# Now all the data cleaning completed. we will start in to analysis

In [ ]:
# 1) Write a query to get Total films released ?
df_bo.select("FilmID").count()

In [45]:
# 2) Write a query to get Total budget ?
df_bo.agg(sum("Budget_in_Crores").alias("TTLBudget")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 47, Finished, Available, Finished)

+---------+
|TTLBudget|
+---------+
|  34653.0|
+---------+



In [46]:
# 3) Write a query to get Total worldwide collection ?
df_bo.agg(sum("Worldwide_Collection_in_Crores").alias("TTLWW_Collection")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 48, Finished, Available, Finished)

+-----------------+
| TTLWW_Collection|
+-----------------+
|79878.78999999998|
+-----------------+



In [47]:
# 4)Write a query to get Total First day collection worldwide ?
df_bo.agg(sum("First_Day_Collection_Worldwide_in_Crores").alias("TTL1stDay_Colelctions")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 49, Finished, Available, Finished)

+---------------------+
|TTL1stDay_Colelctions|
+---------------------+
|   11070.270000000006|
+---------------------+



In [48]:
# 5)Write a query to get Total Overseas collection ?
df_bo.agg(sum("Overseas_Collection_in_Crores").alias("TTL_OS_Collections")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 50, Finished, Available, Finished)

+------------------+
|TTL_OS_Collections|
+------------------+
| 20953.07000000001|
+------------------+



In [49]:
# 6)Write a query to get Total India Gross collection ?
df_bo.agg(sum("India_Gross_Collection_in_Crores").alias("TTL_IND_GRS_Collections")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 51, Finished, Available, Finished)

+-----------------------+
|TTL_IND_GRS_Collections|
+-----------------------+
|      60064.10000000004|
+-----------------------+



In [50]:
# 7)Top 10 filmsbased on world wide collections. Display films,collections
df_bo.select("Title","Worldwide_Collection_in_Crores").sort(desc("Worldwide_Collection_in_Crores")).limit(10).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 52, Finished, Available, Finished)

+--------------------------+------------------------------+
|Title                     |Worldwide_Collection_in_Crores|
+--------------------------+------------------------------+
|Dangal                    |2122.3                        |
|Bãhubali 2: The Conclusion|1788.0                        |
|RRR (Rise Roar Revolt)    |1230.0                        |
|K.G.F: Chapter 2          |1215.0                        |
|Jawan                     |1160.0                        |
|Pathaan                   |1055.0                        |
|Kalki 2898-AD             |1042.25                       |
|Bajrangi Bhaijaan         |922.1                         |
|Animal                    |915.0                         |
|Secret Superstar          |912.6                         |
+--------------------------+------------------------------+



In [51]:
# 8) Total Number of films released by year
df_bo.groupBy(year("Release_Date").alias("Year")).agg(count("FilmID").alias("Films_Count")).sort(desc("Films_Count")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 53, Finished, Available, Finished)

+----+-----------+
|Year|Films_Count|
+----+-----------+
|2022|         80|
|2023|         79|
|2019|         74|
|2017|         72|
|2018|         71|
|2024|         65|
|2016|         41|
|2015|         32|
|2014|         32|
|2021|         31|
|2020|         27|
+----+-----------+



In [52]:
# 9)Top 10 filmsbased on india collections.Display films,collections
df_bo.groupBy("Title").agg(sum("India_Gross_Collection_in_Crores").alias("TTL_IND_GRS_Collections"))\
     .sort(desc("TTL_IND_GRS_Collections")).limit(10).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 54, Finished, Available, Finished)

+--------------------------+-----------------------+
|Title                     |TTL_IND_GRS_Collections|
+--------------------------+-----------------------+
|Bãhubali 2: The Conclusion|1417.0                 |
|K.G.F: Chapter 2          |1000.85                |
|RRR (Rise Roar Revolt)    |915.85                 |
|Kalki 2898-AD             |767.25                 |
|Jawan                     |760.0                  |
|Stree 2: Sarkate Ka Aatank|713.07                 |
|Animal                    |660.0                  |
|Pathaan                   |657.5                  |
|Gadar 2                   |620.5                  |
|Dangal                    |587.0                  |
+--------------------------+-----------------------+



In [53]:
#10)Top 10 filmsbased on overses collections.Display films,collections
df_bo.groupBy("Title").agg(sum("Overseas_Collection_in_Crores").alias("TTL_Overseas_Collections"))\
     .sort(desc("TTL_Overseas_Collections")).limit(10).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 55, Finished, Available, Finished)

+--------------------------+------------------------+
|Title                     |TTL_Overseas_Collections|
+--------------------------+------------------------+
|Dangal                    |1535.3                  |
|Secret Superstar          |831.4                   |
|Bajrangi Bhaijaan         |489.7                   |
|Jawan                     |400.0                   |
|Pathaan                   |397.5                   |
|Bãhubali 2: The Conclusion|371.0                   |
|RRR (Rise Roar Revolt)    |314.15                  |
|PK                        |303.0                   |
|Kalki 2898-AD             |275.0                   |
|Animal                    |255.0                   |
+--------------------------+------------------------+



In [54]:
#11)Top 10 filmsbased on firstday collections.Display films,collections
df_bo.groupBy("Title").agg(sum("First_Day_Collection_Worldwide_in_Crores").alias("TTL_1st_day_WW_Collections"))\
     .sort(desc("TTL_1st_day_WW_Collections")).limit(10).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 56, Finished, Available, Finished)

+--------------------------+--------------------------+
|Title                     |TTL_1st_day_WW_Collections|
+--------------------------+--------------------------+
|RRR (Rise Roar Revolt)    |223.0                     |
|Bãhubali 2: The Conclusion|217.0                     |
|Kalki 2898-AD             |177.7                     |
|K.G.F: Chapter 2          |159.0                     |
|Salaar: Part 1 - Ceasefire|158.1                     |
|Leo                       |142.7                     |
|Devara: Part 1            |142.0                     |
|Jawan                     |129.1                     |
|Adipurush                 |127.5                     |
|Animal                    |116.0                     |
+--------------------------+--------------------------+



In [55]:
# adding weekname column to df_bo dataframe
df_bo = df_bo.withColumn("WeekName", date_format("Release_Date", "EEEE"))

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 57, Finished, Available, Finished)

In [56]:
display(df_bo.head(5))

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 58, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c5ed6d3d-d5b2-4785-ab24-73638ac13613)

In [57]:
#12) Weekday wise films released,Display week name and no of films released
df_bo.groupBy("WeekName").agg(count("FilmID").alias("FilmCount")).sort(desc("FilmCount")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 59, Finished, Available, Finished)

+---------+---------+
| WeekName|FilmCount|
+---------+---------+
|   Friday|      423|
| Thursday|      124|
|Wednesday|       35|
| Saturday|       10|
|   Sunday|        7|
|  Tuesday|        5|
+---------+---------+



In [58]:
#13)Write a query to get OTT platofrm wise movies count
df_bo.groupBy("OTT_Platform").agg(count("FilmID").alias("FilmCount")).sort(desc("FilmCount")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 60, Finished, Available, Finished)

+------------------+---------+
|      OTT_Platform|FilmCount|
+------------------+---------+
|Amazon Prime Video|      384|
|           Netflix|       99|
|   Disney+ Hotstar|       54|
|              ZEE5|       39|
|           Sun NXT|       11|
|               Aha|        8|
|           SonyLIV|        4|
|         JioCinema|        3|
|              Voot|        1|
|Google Play Movies|        1|
+------------------+---------+



In [59]:
#14)Top 10 Directors by films released
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner").groupBy("Director").agg(count("FilmID").alias("FilmCount"))\
     .sort(desc("FilmCount")).limit(10).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 61, Finished, Available, Finished)

+------------------+---------+
|          Director|FilmCount|
+------------------+---------+
|Trivikram Srinivas|        6|
|      Rohit Shetty|        6|
|     Anil Ravipudi|        5|
|     Koratala Siva|        5|
|   A.R. Murugadoss|        5|
|    Boyapati Srinu|        5|
|              Siva|        5|
|    Maruthi Dasari|        4|
|  Lokesh Kanagaraj|        4|
|   Siddharth Anand|        4|
+------------------+---------+



In [60]:
#15)Top 10 directors by world wide collection
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner").groupBy("Director")\
     .agg(sum("Worldwide_Collection_in_Crores").alias("TTL_WW_Collections")).sort(desc("TTL_WW_Collections")).limit(10).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 62, Finished, Available, Finished)

+-------------------+------------------+
|           Director|TTL_WW_Collections|
+-------------------+------------------+
|     S.S. Rajamouli|            3668.0|
|    Siddharth Anand|           2229.45|
|      Nitesh Tiwari|            2122.3|
|     Prashanth Neel|           2101.75|
|              Atlee|           1865.35|
|    Rajkumar Hirani|            1834.5|
|       Rohit Shetty|            1656.6|
|   Lokesh Kanagaraj|1350.1299999999999|
|Sandeep Reddy Vanga|            1342.0|
|         Kabir Khan|           1220.55|
+-------------------+------------------+



In [61]:
#16)Top10 lead actors by world wide collection
df_bo.groupBy("Lead_Actor/Actress").agg(sum("Worldwide_Collection_in_Crores").alias("TTL_WW_Collections"))\
     .sort(desc("TTL_WW_Collections")).limit(10).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 63, Finished, Available, Finished)

+------------------+------------------+
|Lead_Actor/Actress|TTL_WW_Collections|
+------------------+------------------+
|           Prabhas|            5091.5|
|       Salman Khan|            4515.2|
|    Shah Rukh Khan|3800.1000000000004|
|      Akshay Kumar|           3168.22|
|      Joseph Vijay|3076.1099999999997|
|        Aamir Khan|            2914.3|
|       Rajinikanth|           2695.55|
|        Ajay Devgn|2571.2100000000005|
| N.T. Rama Rao Jr.|            2242.6|
|     Ranbir Kapoor|           2111.25|
+------------------+------------------+



In [62]:
#17)Top 10 movies by IMDb rating
df_bo.select("Title","IMDb_Rating").sort(desc("IMDb_Rating")).limit(10).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 64, Finished, Available, Finished)

+--------------------+-----------+
|               Title|IMDb_Rating|
+--------------------+-----------+
|           12th Fail|        8.8|
|         777 Charlie|        8.7|
|Rocketry: The Nam...|        8.7|
|   The Kashmir Files|        8.6|
|  Kishkindha Kaandam|        8.6|
|                  96|        8.5|
|          Sita Ramam|        8.5|
|   Kumbalangi Nights|        8.5|
|            Maharaja|        8.5|
|         Meiyazhagan|        8.5|
+--------------------+-----------+



In [63]:
#18)Bottom 10 movies by IMDb rating
df_bo.select("Title","IMDb_Rating").sort(asc("IMDb_Rating")).limit(10).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 65, Finished, Available, Finished)

+--------------------+-----------+
|               Title|IMDb_Rating|
+--------------------+-----------+
|              Race 3|        1.9|
|            Baaghi 3|        2.2|
|Student of the Ye...|        2.2|
|         Heropanti 2|        2.3|
|      Chandramukhi 2|        2.6|
|               Liger|        2.6|
|           Adipurush|        2.7|
|              Gunday|        2.7|
|           Dabangg 3|        3.0|
|       A Flying Jatt|        3.1|
+--------------------+-----------+



In [64]:
#19)Write a query to get 5 longest run time movies
df_bo.select("Title","Runtime_(mins)").sort(desc("Runtime_(mins)")).limit(10).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 66, Finished, Available, Finished)

+----------------------------+--------------+
|Title                       |Runtime_(mins)|
+----------------------------+--------------+
|Animal                      |204           |
|I                           |188           |
|RRR (Rise Roar Revolt)      |187           |
|Avane Srimannarayana        |186           |
|Kurukshetra                 |185           |
|Jilla                       |185           |
|M.S. Dhoni: The Untold Story|184           |
|Cobra                       |183           |
|The GOAT                    |183           |
|Kaatera                     |183           |
+----------------------------+--------------+



In [65]:
#20)Write a query to get 5 shortest run time movies
df_bo.select("Title","Runtime_(mins)").sort(asc("Runtime_(mins)")).limit(10).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 67, Finished, Available, Finished)

+----------------------------------+--------------+
|Title                             |Runtime_(mins)|
+----------------------------------+--------------+
|Kill                              |105           |
|Raksha Bandhan                    |108           |
|Bhoot: Part One - The Haunted Ship|114           |
|Hichki                            |116           |
|The Ghazi Attack                  |116           |
|Guru                              |116           |
|Bareilly Ki Barfi                 |116           |
|Raju Gari Gadhi 2                 |117           |
|Shubh Mangal Zyada Saavdhan       |117           |
|Badla                             |118           |
+----------------------------------+--------------+



In [66]:
#21)Top7 movies by world wide collection in Bollywood
df_bo.filter(col("Industry") == "Bollywood").select("Title","Worldwide_Collection_in_Crores")\
     .sort(desc("Worldwide_Collection_in_Crores")).limit(7).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 68, Finished, Available, Finished)

+--------------------+------------------------------+
|               Title|Worldwide_Collection_in_Crores|
+--------------------+------------------------------+
|              Dangal|                        2122.3|
|               Jawan|                        1160.0|
|             Pathaan|                        1055.0|
|   Bajrangi Bhaijaan|                         922.1|
|              Animal|                         915.0|
|    Secret Superstar|                         912.6|
|Stree 2: Sarkate ...|                        857.07|
+--------------------+------------------------------+



In [67]:
#22)Top7 movies by world wide collection in Tollywood
df_bo.filter(col("Industry") == "Tollywood").select("Title","Worldwide_Collection_in_Crores")\
     .sort(desc("Worldwide_Collection_in_Crores")).limit(7).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 69, Finished, Available, Finished)

+--------------------------+------------------------------+
|Title                     |Worldwide_Collection_in_Crores|
+--------------------------+------------------------------+
|Bãhubali 2: The Conclusion|1788.0                        |
|RRR (Rise Roar Revolt)    |1230.0                        |
|Kalki 2898-AD             |1042.25                       |
|Bãhubali: The Beginning   |650.0                         |
|Salaar: Part 1 - Ceasefire|617.75                        |
|Saaho                     |451.0                         |
|Devara: Part 1            |421.6                         |
+--------------------------+------------------------------+



In [68]:
#23) Top7 movies by world wide collection in Kollywood
df_bo.filter(col("Industry") == "Kollywood").select("Title","Worldwide_Collection_in_Crores")\
     .sort(desc("Worldwide_Collection_in_Crores")).limit(7).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 70, Finished, Available, Finished)

+-------------------------+------------------------------+
|Title                    |Worldwide_Collection_in_Crores|
+-------------------------+------------------------------+
|2                        |701.0                         |
|Leo                      |605.9                         |
|Jailer                   |604.5                         |
|Ponniyin Selvan: Part One|488.36                        |
|The GOAT                 |451.23                        |
|Vikram                   |414.43                        |
|Ponniyin Selvan: Part Two|344.63                        |
+-------------------------+------------------------------+



In [69]:
#24)Top7 movies by world wide collection in Sandalwood
df_bo.filter(col("Industry") == "Sandalwood").select("Title","Worldwide_Collection_in_Crores")\
     .sort(desc("Worldwide_Collection_in_Crores")).limit(7).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 71, Finished, Available, Finished)

+-----------------+------------------------------+
|Title            |Worldwide_Collection_in_Crores|
+-----------------+------------------------------+
|K.G.F: Chapter 2 |1215.0                        |
|Kantara          |407.82                        |
|K.G.F: Chapter 1 |238.0                         |
|777 Charlie      |102.75                        |
|VR (Vikrant Rona)|100.35                        |
|James            |94.2                          |
|Kurukshetra      |81.0                          |
+-----------------+------------------------------+



In [70]:
#25) Top7 movies by world wide collection in Mollywood
df_bo.filter(col("Industry") == "Mollywood").select("Title","Worldwide_Collection_in_Crores")\
     .sort(desc("Worldwide_Collection_in_Crores")).limit(7).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 72, Finished, Available, Finished)

+--------------+------------------------------+
|Title         |Worldwide_Collection_in_Crores|
+--------------+------------------------------+
|Manjummel Boys|241.1                         |
|2018          |180.03                        |
|The Goat Life |157.35                        |
|Aavesham      |154.79                        |
|Pulimurugan   |150.0                         |
|Premalu       |131.18                        |
|Lucifer       |129.2                         |
+--------------+------------------------------+



In [71]:
#26) Write query to display industry and verdict wise films count
df_bo.groupBy("Industry","verdict").agg(count("FilmID").alias("FilmCount")).sort(asc("Industry")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 73, Finished, Available, Finished)

+---------+--------------------+---------+
| Industry|             verdict|FilmCount|
+---------+--------------------+---------+
|Bollywood|                 Hit|       39|
|Bollywood|         Blockbuster|       30|
|Bollywood|                Flop|       22|
|Bollywood|       Above Average|        7|
|Bollywood|           Super Hit|       22|
|Bollywood|            Disaster|       25|
|Bollywood|       Below Average|       14|
|Bollywood|All Time Blockbuster|        5|
|Bollywood|             Average|       18|
|Kollywood|           Super Hit|       23|
|Kollywood|                Flop|       12|
|Kollywood|                 Hit|       16|
|Kollywood|             Average|        8|
|Kollywood|       Below Average|       19|
|Kollywood|       Above Average|        5|
|Kollywood|         Blockbuster|       44|
|Kollywood|            Disaster|       13|
|Mollywood|            Disaster|        5|
|Mollywood|                Flop|        2|
|Mollywood|                 Hit|       10|
+---------+

In [72]:
#27) Write query to get films based on budget in Bollowood. display title and budget
df_bo.filter(col("Industry") == "Bollywood").select("Title","Budget_in_Crores").show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 74, Finished, Available, Finished)

+--------------------+----------------+
|               Title|Budget_in_Crores|
+--------------------+----------------+
|               Sanju|           100.0|
|              Simmba|           125.0|
|Tanu Weds Manu Re...|            40.0|
|          Ek Villain|            40.0|
|           Padmaavat|           190.0|
|              Sultan|           140.0|
|            Baaghi 2|            65.0|
|               Stree|            25.0|
|Sonu Ke Titu Ki S...|            25.0|
|              Baaghi|            35.0|
|    Secret Superstar|            50.0|
|     Tiger Zinda Hai|           200.0|
|        Hindi Medium|            25.0|
|          Drishyam 2|            65.0|
|   The Kashmir Files|            20.0|
|Tanhaji: The Unsu...|           150.0|
|         Kabir Singh|            55.0|
|Uri: The Surgical...|            40.0|
|      Mission Mangal|            75.0|
|               Badla|            20.0|
+--------------------+----------------+
only showing top 20 rows



In [73]:
#28) Write query to get films based on budget in Tollywood
df_bo.filter(col("Industry") == "Tollywood").select("Title","Budget_in_Crores").show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 75, Finished, Available, Finished)

+--------------------+----------------+
|               Title|Budget_in_Crores|
+--------------------+----------------+
|         Srimanthudu|            70.0|
|     Geetha Govindam|             8.5|
|       Mahanubhavudu|            15.0|
|         Race Gurram|            50.0|
|          Nenu Local|            20.0|
|               Manam|            25.0|
|              Rx 100|             2.0|
|Bhale Bhale Magad...|            10.0|
|            Mahanati|            20.0|
|  Shatamanam Bhavati|            15.0|
|               Chalo|             5.0|
|          Goodachari|             5.0|
|RRR (Rise Roar Re...|           550.0|
|               Fidaa|            15.0|
|          Ninnu Kori|            20.0|
|         Arjun Reddy|             5.0|
|               Krack|            35.0|
|              Uppena|            25.0|
|       Jathi Ratnalu|            15.0|
|        Karthikeya 2|            25.0|
+--------------------+----------------+
only showing top 20 rows



In [74]:
#29) Write query to get films based on budget in Kollywood
df_bo.filter(col("Industry") == "Kollywood").select("Title","Budget_in_Crores").show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 76, Finished, Available, Finished)

+--------------------+----------------+
|               Title|Budget_in_Crores|
+--------------------+----------------+
|      Janatha Garage|            50.0|
|             Kaththi|            70.0|
|   Kadaikutty Singam|            25.0|
|       Irumbu Thirai|            20.0|
|              Veeram|            45.0|
|    Imaikkaa Nodigal|            10.0|
|          Badhaai ho|            30.0|
|Velaiilla Pattadhari|            20.0|
|                  96|            15.0|
|            Ratsasan|            10.0|
|     Thamizh Padam 2|            10.0|
|     Kolamavu Kokila|            10.0|
|                   I|            90.0|
|              Doctor|            40.0|
|              Mersal|           125.0|
|        Thani Oruvan|            20.0|
|        Vikram Vedha|            15.0|
|               Kavan|            10.0|
|          Kanchana 2|            17.0|
|            Maanaadu|            35.0|
+--------------------+----------------+
only showing top 20 rows



In [75]:
#30)Write query to get films based on budget in Sandalwood
df_bo.filter(col("Industry") == "Sandalwood").select("Title","Budget_in_Crores").show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 77, Finished, Available, Finished)

+--------------------+----------------+
|               Title|Budget_in_Crores|
+--------------------+----------------+
|            Maanikya|            15.0|
|              Ugramm|             4.0|
|              Tagaru|            10.0|
|             Hebbuli|            20.0|
|         777 Charlie|            15.0|
|Avane Srimannarayana|            25.0|
|            Yajamana|            20.0|
|         Raajakumara|            20.0|
|         Yuvarathnaa|            25.0|
|    K.G.F: Chapter 1|            50.0|
|    K.G.F: Chapter 2|           100.0|
|         The Villain|            45.0|
|Mr. And Mrs. Ramc...|             6.0|
|         Anjaniputra|            20.0|
|        Chakravarthy|            20.0|
|             Roberrt|            30.0|
|              Pogaru|            25.0|
|         Kotigobba 3|            25.0|
|              Salaga|            10.0|
|             Kantara|            16.0|
+--------------------+----------------+
only showing top 20 rows



In [76]:
#31) Write query to get films based on budget in Mollywood
df_bo.filter(col("Industry") == "Mollywood").select("Title","Budget_in_Crores").show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 78, Finished, Available, Finished)

+--------------------+----------------+
|               Title|Budget_in_Crores|
+--------------------+----------------+
|Abrahaminte Santh...|            10.0|
|      Bheeshmaparvam|            18.0|
|      Njan Prakashan|             8.0|
|               Aadhi|             7.0|
|            Varathan|             3.5|
| Sudani from Nigeria|             2.0|
|Munthirivallikal ...|            10.0|
|                Ezra|             8.0|
|           Ramaleela|            14.0|
|              Aadu 2|             7.5|
|     Anjaam Pathiraa|             5.0|
|            Take Off|             8.0|
|  Ennu Ninte Moideen|            10.0|
|       Two Countries|            12.0|
|  Amar Akbar Anthony|             7.0|
|Kattappanayile Ri...|             3.0|
|               Oppam|             7.0|
|    Action Hero Biju|             3.0|
|            Hridayam|             7.0|
|        Malikappuram|             4.0|
+--------------------+----------------+
only showing top 20 rows



In [77]:
#32) Top 5 movies by IMDb rating from Bollywood
df_bo.filter(col("Industry") == "Bollywood").select("Title","IMDb_Rating").sort(desc("IMDb_Rating")).limit(5).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 79, Finished, Available, Finished)

+-----------------+-----------+
|            Title|IMDb_Rating|
+-----------------+-----------+
|        12th Fail|        8.8|
|The Kashmir Files|        8.6|
|           Sachin|        8.5|
|  Laapataa Ladies|        8.4|
|           Dangal|        8.3|
+-----------------+-----------+



In [78]:
#33)Top 5 movies by IMDb rating from  Kollywood
df_bo.filter(col("Industry") == "Kollywood").select("Title","IMDB_Rating").sort(desc("IMDB_Rating")).limit(5).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 80, Finished, Available, Finished)

+--------------------+-----------+
|               Title|IMDB_Rating|
+--------------------+-----------+
|Rocketry: The Nam...|        8.7|
|                  96|        8.5|
|            Maharaja|        8.5|
|         Meiyazhagan|        8.5|
|              Asuran|        8.4|
+--------------------+-----------+



In [79]:
#34)Top 5 movies by IMDb rating from Tollywood
df_bo.filter(col("Industry") == "Tollywood").select("Title","IMDB_Rating").sort(desc("IMDB_Rating")).limit(5).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 81, Finished, Available, Finished)

+------------+-----------+
|       Title|IMDB_Rating|
+------------+-----------+
|  Sita Ramam|        8.5|
|      Jersey|        8.5|
|    Mahanati|        8.4|
|Rangasthalam|        8.2|
|    Hi Nanna|        8.2|
+------------+-----------+



In [80]:
#35)Top 5 movies by IMDb rating from Sandalwood
df_bo.filter(col("Industry") == "Sandalwood").select("Title","IMDB_Rating").sort(desc("IMDB_Rating")).limit(5).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 82, Finished, Available, Finished)

+--------------------+-----------+
|               Title|IMDB_Rating|
+--------------------+-----------+
|         777 Charlie|        8.7|
|    K.G.F: Chapter 2|        8.2|
|    K.G.F: Chapter 1|        8.2|
|             Kantara|        8.2|
|Sapta Sagaradaach...|        8.2|
+--------------------+-----------+



In [81]:
#36) Top 5 movies by IMDb rating from Mollywood
df_bo.filter(col("Industry") == "Mollywood").select("Title","IMDb_Rating").sort(desc("IMDb_Rating")).limit(5).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 83, Finished, Available, Finished)

+------------------+-----------+
|             Title|IMDb_Rating|
+------------------+-----------+
|Kishkindha Kaandam|        8.6|
| Kumbalangi Nights|        8.5|
|    Bangalore Days|        8.3|
|            Premam|        8.3|
|    Jana Gana Mana|        8.3|
+------------------+-----------+



In [82]:
#37) Write a query to get language wise budget?
df_bo.join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner").groupBy("Language")\
     .agg(sum("Budget_in_Crores").alias("TTL_Budget")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 84, Finished, Available, Finished)

+---------+----------+
| Language|TTL_Budget|
+---------+----------+
|  Kannada|     949.0|
|   Telugu|    9144.5|
|    Tamil|    8682.0|
|    Hindi|   14535.0|
|Malayalam|    1342.5|
+---------+----------+



In [83]:
#38) Write a query to get language wise how many directors are there?
df_bo.join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner").groupBy("Language")\
     .agg(count("DirectorID").alias("Director_Count")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 85, Finished, Available, Finished)

+---------+--------------+
| Language|Director_Count|
+---------+--------------+
|  Kannada|            32|
|   Telugu|           166|
|    Tamil|           140|
|    Hindi|           182|
|Malayalam|            84|
+---------+--------------+



In [84]:
#39) Write a query to get language wise worldwide total collection ?
df_bo.join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner").groupBy("Language")\
     .agg(sum("Worldwide_Collection_in_Crores").alias("TTL_WW_Collections")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 86, Finished, Available, Finished)

+---------+------------------+
| Language|TTL_WW_Collections|
+---------+------------------+
|  Kannada|3369.4199999999996|
|   Telugu|18065.270000000004|
|    Tamil|15235.989999999998|
|    Hindi| 38610.79000000001|
|Malayalam|           4597.32|
+---------+------------------+



In [85]:
#40) Write a query to get language, lead actor/actress wise films they acted?
df_bo.join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner").groupBy("Language","Lead_Actor/Actress")\
     .agg(count("FilmID").alias("Film_Count")).sort(desc("Film_Count")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 87, Finished, Available, Finished)

+---------+--------------------+----------+
| Language|  Lead_Actor/Actress|Film_Count|
+---------+--------------------+----------+
|    Hindi|        Akshay Kumar|        20|
|    Hindi|          Ajay Devgn|        14|
|Malayalam|           Mammootty|        13|
|    Tamil|             Dhanush|        13|
|    Tamil|        Joseph Vijay|        12|
|    Hindi|         Salman Khan|        12|
|   Telugu|                Nani|        12|
|    Tamil|     Sivakarthikeyan|        11|
|Malayalam|            Mohanlal|        11|
|   Telugu|         Mahesh Babu|        10|
|   Telugu|           Ravi Teja|        10|
|    Tamil|         Rajinikanth|        10|
|    Tamil|              Karthi|        10|
|Malayalam|Prithviraj Sukumaran|         9|
|    Tamil|              Vikram|         9|
|   Telugu|   Vijay Deverakonda|         9|
|    Tamil|         Ajith Kumar|         8|
|    Tamil|              Suriya|         8|
|    Hindi|      Shah Rukh Khan|         8|
|   Telugu|             Prabhas|

In [86]:
#41) Write a query to get language, year wise films released?
df_bo.join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner").groupBy("Language",year("Release_Date").alias("Year"))\
     .agg(count("FilmID").alias("Film_Count")).sort(desc("Film_Count")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 88, Finished, Available, Finished)

+---------+----+----------+
| Language|Year|Film_Count|
+---------+----+----------+
|   Telugu|2023|        25|
|    Hindi|2022|        24|
|    Hindi|2017|        24|
|   Telugu|2017|        24|
|    Hindi|2023|        23|
|   Telugu|2018|        23|
|    Tamil|2018|        22|
|   Telugu|2022|        22|
|    Hindi|2024|        20|
|    Tamil|2019|        19|
|   Telugu|2019|        19|
|    Hindi|2016|        19|
|    Tamil|2023|        18|
|    Hindi|2019|        17|
|    Tamil|2022|        17|
|    Tamil|2024|        16|
|    Hindi|2018|        15|
|Malayalam|2024|        14|
|   Telugu|2024|        14|
|    Hindi|2014|        14|
+---------+----+----------+
only showing top 20 rows



In [87]:
#42)Write a query to get films which was not released on overseas?
df_bo.filter(col("Overseas_Collection_in_Crores") == 0).select("Title").show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 89, Finished, Available, Finished)

+------------+
|       Title|
+------------+
|    Maanikya|
|      Ugramm|
|      Tagaru|
|     Hebbuli|
| Anjaniputra|
|Chakravarthy|
|         118|
+------------+



In [88]:
#43) Write a query to get language wise top 3 longest runtime moves?
df_bo.join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner").select("Language","Title","Runtime_(mins)")\
     .withColumn("Rank",row_number().over(Window.partitionBy("Language").orderBy(desc("Runtime_(mins)"))))\
     .filter(col("Rank") <=3).select("Language","Title","Runtime_(mins)").show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 90, Finished, Available, Finished)

+---------+---------------------------------+--------------+
|Language |Title                            |Runtime_(mins)|
+---------+---------------------------------+--------------+
|Hindi    |Animal                           |204           |
|Hindi    |M.S. Dhoni: The Untold Story     |184           |
|Hindi    |Maidaan                          |181           |
|Kannada  |Avane Srimannarayana             |186           |
|Kannada  |Kurukshetra                      |185           |
|Kannada  |Kaatera                          |183           |
|Malayalam|Marakkar: Lion of the Arabian Sea|181           |
|Malayalam|Ayyappanum Koshiyum              |177           |
|Malayalam|Lucifer                          |175           |
|Tamil    |I                                |188           |
|Tamil    |Jilla                            |185           |
|Tamil    |Cobra                            |183           |
|Telugu   |RRR (Rise Roar Revolt)           |187           |
|Telugu   |Arjun Reddy  

In [89]:
#44) Write a query to get language wise bottom 5 shortest runtime moves?
df_bo.join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner").select("Language","Title","Runtime_(mins)")\
     .withColumn("Rank",row_number().over(Window.partitionBy("Language").orderBy(asc("Runtime_(mins)"))))\
     .filter(col("Rank") <= 3).select("Language","Title","Runtime_(mins)").show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 91, Finished, Available, Finished)

+---------+----------------------------------+--------------+
|Language |Title                             |Runtime_(mins)|
+---------+----------------------------------+--------------+
|Hindi    |Kill                              |105           |
|Hindi    |Raksha Bandhan                    |108           |
|Hindi    |Bhoot: Part One - The Haunted Ship|114           |
|Kannada  |Salaga                            |124           |
|Kannada  |Tagaru                            |129           |
|Kannada  |Ugramm                            |132           |
|Malayalam|Malikappuram                      |121           |
|Malayalam|Sudani from Nigeria               |123           |
|Malayalam|Vaazha: Biopic of a Billion Boys  |125           |
|Tamil    |Kochadaiiyaan                     |118           |
|Tamil    |Dhilluku Dhuddu 2                 |119           |
|Tamil    |Kadaram Kondan                    |121           |
|Telugu   |The Ghazi Attack                  |116           |
|Telugu 

In [90]:
#45) Write a query to get language wise top 5 films based first day collections?
df_bo.join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner")\
     .select("Language","Title",col("First_Day_Collection_Worldwide_in_Crores").alias("TTL_1st_D_Collections"))\
     .withColumn("Rank",row_number().over(Window.partitionBy("Language").orderBy(desc("TTL_1st_D_Collections"))))\
     .filter(col("Rank") <= 5).select("Language","Title","TTL_1st_D_Collections").show(30,truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 92, Finished, Available, Finished)

+---------+---------------------------------+---------------------+
|Language |Title                            |TTL_1st_D_Collections|
+---------+---------------------------------+---------------------+
|Hindi    |Jawan                            |129.1                |
|Hindi    |Animal                           |116.0                |
|Hindi    |Pathaan                          |104.8                |
|Hindi    |Tiger 3                          |94.0                 |
|Hindi    |Stree 2: Sarkate Ka Aatank       |80.0                 |
|Kannada  |K.G.F: Chapter 2                 |159.0                |
|Kannada  |VR (Vikrant Rona)                |25.5                 |
|Kannada  |K.G.F: Chapter 1                 |25.0                 |
|Kannada  |James                            |23.5                 |
|Kannada  |Roberrt                          |16.9                 |
|Malayalam|Marakkar: Lion of the Arabian Sea|20.3                 |
|Malayalam|Odiyan                           |18.

In [91]:
#46)Write a query to get language wise top 5 films based India gross collections? 
df_bo.join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner")\
     .select("Language","Title", col("India_Gross_Collection_in_Crores").alias("Ind_Gross_Collections"))\
     .withColumn("Rank",row_number().over(Window.partitionBy("Language").orderBy(desc("Ind_Gross_Collections"))))\
     .filter(col("Rank") <= 5).select("Language","Title","Ind_Gross_Collections").show(30,truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 93, Finished, Available, Finished)

+---------+--------------------------+---------------------+
|Language |Title                     |Ind_Gross_Collections|
+---------+--------------------------+---------------------+
|Hindi    |Jawan                     |760.0                |
|Hindi    |Stree 2: Sarkate Ka Aatank|713.07               |
|Hindi    |Animal                    |660.0                |
|Hindi    |Pathaan                   |657.5                |
|Hindi    |Gadar 2                   |620.5                |
|Kannada  |K.G.F: Chapter 2          |1000.85              |
|Kannada  |Kantara                   |363.82               |
|Kannada  |K.G.F: Chapter 1          |228.0                |
|Kannada  |777 Charlie               |96.95                |
|Kannada  |VR (Vikrant Rona)         |94.75                |
|Malayalam|Manjummel Boys            |167.65               |
|Malayalam|2018                      |110.53               |
|Malayalam|Pulimurugan               |105.9                |
|Malayalam|Aavesham     

In [92]:
#47)Write a query to get language, Director wise films count?
df_bo.join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner")\
     .join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .groupBy("Language","Director").agg(count("FilmID").alias("Film_Count")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 94, Finished, Available, Finished)

+---------+--------------------+----------+
| Language|            Director|Film_Count|
+---------+--------------------+----------+
|    Hindi|          Raja Menon|         1|
|    Hindi|Aniruddha Roy Cho...|         1|
|    Hindi|Gauri Joshi, Raaj...|         1|
|    Hindi|        Anand Tiwari|         1|
|    Tamil|        P.S. Mithran|         3|
|    Tamil|           Sundar C.|         4|
|   Telugu|    Saagar K Chandra|         1|
|Malayalam|Ratheesh Balakris...|         1|
|Malayalam|          G. Prajith|         1|
|   Telugu|Radha Krishna Jag...|         2|
|    Hindi|Vijay Krishna Ach...|         1|
|  Kannada|      Prashanth Neel|         3|
|  Kannada|          Kiranraj K|         1|
|   Telugu|    Gowtam Tinnanuri|         1|
|   Telugu|          K.V. Guhan|         1|
|   Telugu|       Vivek Athreya|         2|
|    Hindi|     Siddharth Anand|         4|
|   Telugu|        Venky Atluri|         3|
|    Hindi|         Abhinay Deo|         1|
|Malayalam|   Dhyan Sreenivasan|

In [93]:
#48)  Write a query to get language wise OTT platofrm wise fims count?
df_bo.join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner")\
     .groupBy("Language","OTT_Platform")\
     .agg(count("FilmID").alias("FilmCount")).show()     

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 95, Finished, Available, Finished)

+---------+------------------+---------+
| Language|      OTT_Platform|FilmCount|
+---------+------------------+---------+
|  Kannada|              Voot|        1|
|    Tamil|           Netflix|       19|
|    Hindi|         JioCinema|        3|
|    Tamil|Amazon Prime Video|      100|
|   Telugu|Google Play Movies|        1|
|    Hindi|               Aha|        1|
|  Kannada|   Disney+ Hotstar|        1|
|    Tamil|               Aha|        1|
|   Telugu|           Netflix|       23|
|Malayalam|              ZEE5|        3|
|    Tamil|   Disney+ Hotstar|        9|
|Malayalam|               Aha|        2|
|    Hindi|           Netflix|       52|
|Malayalam|           Netflix|        5|
|    Hindi|Amazon Prime Video|       78|
|Malayalam|   Disney+ Hotstar|        7|
|    Tamil|           Sun NXT|        4|
|   Telugu|Amazon Prime Video|      119|
|Malayalam|           Sun NXT|        7|
|Malayalam|Amazon Prime Video|       60|
+---------+------------------+---------+
only showing top

In [94]:
#49) What are top 10 fims based language and first day collection?
df_bo.join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner")\
     .select("Language","Title",col("First_Day_Collection_Worldwide_in_Crores").alias("1st_Day_WW_Collections"))\
     .withColumn("Rank",row_number().over(Window.partitionBy("Language").orderBy(desc("1st_Day_WW_Collections"))))\
     .filter(col("Rank") <= 10).select("Language","Title","1st_Day_WW_Collections").show(50,truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 96, Finished, Available, Finished)

+---------+---------------------------------+----------------------+
|Language |Title                            |1st_Day_WW_Collections|
+---------+---------------------------------+----------------------+
|Hindi    |Jawan                            |129.1                 |
|Hindi    |Animal                           |116.0                 |
|Hindi    |Pathaan                          |104.8                 |
|Hindi    |Tiger 3                          |94.0                  |
|Hindi    |Stree 2: Sarkate Ka Aatank       |80.0                  |
|Hindi    |Thugs of Hindostan               |76.0                  |
|Hindi    |War                              |74.3                  |
|Hindi    |Sultan                           |71.3                  |
|Hindi    |Brahmastra Part One: Shiva       |70.0                  |
|Hindi    |Prem Ratan Dhan Payo             |66.2                  |
|Kannada  |K.G.F: Chapter 2                 |159.0                 |
|Kannada  |VR (Vikrant Rona)      

In [95]:
#50) Write a query to get director wise number of fims released  from year 2017 to 2019
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .select("FilmID","Director",year("Release_Date").alias("Year"))\
     .filter(col("Year").isin(2017,2018,2019))\
     .groupBy("Director")\
     .agg(count("FilmID").alias("FilmCount"))\
     .select("Director","FilmCount").sort(desc("FilmCount")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 97, Finished, Available, Finished)

+--------------------+---------+
|            Director|FilmCount|
+--------------------+---------+
|      Maruthi Dasari|        3|
|      Harish Shankar|        2|
|  Trivikram Srinivas|        2|
|         Mani Ratnam|        2|
|      Satish Vegesna|        2|
|    Sathyan Anthikad|        2|
|     Vikram K. Kumar|        2|
|            Pandiraj|        2|
|Radha Krishna Jag...|        2|
|        Venky Atluri|        2|
|       Jeethu Joseph|        2|
|   Thirumala Kishore|        2|
|               Atlee|        2|
|         Vetrimaaran|        2|
|         Amal Neerad|        2|
|           H. Vinoth|        2|
|Trinadha Rao Nakkina|        2|
|     A.R. Murugadoss|        2|
|        P.S. Mithran|        2|
|      Boyapati Srinu|        2|
+--------------------+---------+
only showing top 20 rows



In [96]:
#51)Write a query to get director wise world wide collections?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .groupBy("Director")\
     .agg(sum("Worldwide_Collection_in_Crores").alias("WW_Collections"))\
     .sort(desc("WW_Collections")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 98, Finished, Available, Finished)

+---------------------+------------------+
|Director             |WW_Collections    |
+---------------------+------------------+
|S.S. Rajamouli       |3668.0            |
|Siddharth Anand      |2229.45           |
|Nitesh Tiwari        |2122.3            |
|Prashanth Neel       |2101.75           |
|Atlee                |1865.35           |
|Rajkumar Hirani      |1834.5            |
|Rohit Shetty         |1656.6            |
|Lokesh Kanagaraj     |1350.1299999999999|
|Sandeep Reddy Vanga  |1342.0            |
|Kabir Khan           |1220.55           |
|Ali Abbas Zafar      |1170.99           |
|Sanjay Leela Bhansali|1158.5            |
|Amar Kaushik         |1133.98           |
|Nag Ashwin           |1125.75           |
|S. Shankar           |1089.83           |
|Advait Chandan       |1046.1            |
|Koratala Siva        |978.6             |
|Nelson Dilipkumar    |952.88            |
|A.R. Murugadoss      |920.6             |
|Trivikram Srinivas   |877.45            |
+----------

In [97]:
#52) Write a query to get director wise first day world wide collections?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .groupBy("Director")\
     .agg(sum("First_Day_Collection_Worldwide_in_Crores").alias("1st_Day_Collections"))\
     .orderBy(desc("1st_Day_Collections")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 99, Finished, Available, Finished)

+---------------------------------------+-------------------+
|Director                               |1st_Day_Collections|
+---------------------------------------+-------------------+
|S.S. Rajamouli                         |513.0              |
|Prashanth Neel                         |344.1              |
|Koratala Siva                          |334.0              |
|Trivikram Srinivas                     |276.3              |
|Atlee                                  |270.2              |
|Lokesh Kanagaraj                       |263.48             |
|Siddharth Anand                        |241.64999999999998 |
|A.R. Murugadoss                        |212.1              |
|Nelson Dilipkumar                      |196.5              |
|S. Shankar                             |196.25             |
|Ali Abbas Zafar                        |183.89999999999998 |
|Nag Ashwin                             |181.2              |
|Rohit Shetty                           |153.17999999999998 |
|Sukumar

In [98]:
#53) Write a query to get director wise India gross collections?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .groupBy("Director")\
     .agg(sum("India_Gross_Collection_in_Crores").alias("Ind_Gross_Collectins"))\
     .orderBy(desc("Ind_Gross_Collectins")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 100, Finished, Available, Finished)

+---------------------+--------------------+
|Director             |Ind_Gross_Collectins|
+---------------------+--------------------+
|S.S. Rajamouli       |2848.85             |
|Prashanth Neel       |1747.6              |
|Siddharth Anand      |1560.9099999999999  |
|Atlee                |1256.05             |
|Rohit Shetty         |1246.2              |
|Rajkumar Hirani      |1199.5              |
|Sandeep Reddy Vanga  |1028.4              |
|Amar Kaushik         |961.48              |
|Lokesh Kanagaraj     |956.8799999999999   |
|Ali Abbas Zafar      |856.79              |
|Nag Ashwin           |840.25              |
|S. Shankar           |826.83              |
|Sanjay Leela Bhansali|818.5               |
|Koratala Siva        |763.0               |
|Trivikram Srinivas   |711.65              |
|A.R. Murugadoss      |699.7               |
|Kabir Khan           |671.5               |
|Nelson Dilipkumar    |669.88              |
|Om Raut              |668.0               |
|Sukumar  

In [99]:
#54)Write a query to get director wise overseas collections?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .groupBy("Director")\
     .agg(sum("Overseas_Collection_in_Crores").alias("Overseas_Collections"))\
     .orderBy(desc("Overseas_Collections")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 101, Finished, Available, Finished)

+---------------------------------------+--------------------+
|Director                               |Overseas_Collections|
+---------------------------------------+--------------------+
|Nitesh Tiwari                          |1535.3              |
|Advait Chandan                         |892.4               |
|Siddharth Anand                        |819.37              |
|S.S. Rajamouli                         |819.15              |
|Rajkumar Hirani                        |635.0               |
|Atlee                                  |609.3               |
|Kabir Khan                             |549.05              |
|Rohit Shetty                           |410.4               |
|Lokesh Kanagaraj                       |393.25              |
|Prashanth Neel                         |354.15              |
|Sanjay Leela Bhansali                  |340.0               |
|Ali Abbas Zafar                        |314.2               |
|Sandeep Reddy Vanga                    |313.6         

In [100]:
#55) Write a query to get director, lead actor/actress and number of films?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .groupBy("Director","Lead_Actor/Actress")\
     .agg(count("FilmID").alias("FilmCount"))\
     .orderBy(asc("Director")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 102, Finished, Available, Finished)

+---------------------+--------------------+---------+
|Director             |Lead_Actor/Actress  |FilmCount|
+---------------------+--------------------+---------+
|A.R. Murugadoss      |Rajinikanth         |1        |
|A.R. Murugadoss      |Joseph Vijay        |2        |
|A.R. Murugadoss      |Mahesh Babu         |1        |
|A.R. Murugadoss      |Akshay Kumar        |1        |
|Aanand L. Rai        |Kangana Ranaut      |1        |
|Aanand L. Rai        |Shah Rukh Khan      |1        |
|Aanand L. Rai        |Akshay Kumar        |1        |
|Aashiq Abu           |Parvathy Thiruvothu |1        |
|Abhilash Joshiy      |Dulquer Salmaan     |1        |
|Abhinay Deo          |John Abraham        |1        |
|Abhishek Nama        |Nandamuri Kalyan Ram|1        |
|Abhishek Pathak      |Ajay Devgn          |1        |
|Abhishek Varman      |Kriti Sanon         |1        |
|Abhishek Varman      |Arjun Kapoor        |1        |
|Abrid Shine          |Nivin Pauly         |1        |
|Adhik Rav

In [101]:
#56)Write a query to get films which is having budget in between 150 crores and 277 crores?
df_bo.select("Title","Budget_in_Crores").filter((col("Budget_in_Crores") >= 150) & (col("Budget_in_Crores") < 277)).show()


StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 103, Finished, Available, Finished)

+--------------------+----------------+
|               Title|Budget_in_Crores|
+--------------------+----------------+
|           Padmaavat|           190.0|
|     Tiger Zinda Hai|           200.0|
|Ponniyin Selvan: ...|           250.0|
|Tanhaji: The Unsu...|           150.0|
|              Animal|           150.0|
|                 Leo|           250.0|
|              Jailer|           200.0|
|                 War|           175.0|
|              Varisu|           200.0|
|               Beast|           150.0|
|              Darbar|           200.0|
|           Annaatthe|           180.0|
|             Valimai|           150.0|
|Ponniyin Selvan: ...|           250.0|
|           Vettaiyan|           200.0|
|            Indian 2|           250.0|
|Pushpa: The Rise ...|           150.0|
|Bãhubali 2: The C...|           250.0|
|Bãhubali: The Beg...|           180.0|
|      Devara: Part 1|           250.0|
+--------------------+----------------+
only showing top 20 rows



In [102]:
#57) Write a query to get director, week name wise films released?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .groupBy("Director","WeekName")\
     .agg(count("FilmID").alias("FilmCount"))\
     .orderBy(desc("FilmCount")).show(truncate=False)


StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 104, Finished, Available, Finished)

+---------------+---------+---------+
|Director       |WeekName |FilmCount|
+---------------+---------+---------+
|Rohit Shetty   |Friday   |6        |
|Mohit Suri     |Friday   |4        |
|Siva           |Friday   |4        |
|Sundar C.      |Friday   |3        |
|Meghna Gulzar  |Friday   |3        |
|P.S. Mithran   |Friday   |3        |
|Sukumar        |Friday   |3        |
|A.R. Murugadoss|Wednesday|3        |
|Anees Bazmee   |Friday   |3        |
|S.S. Rajamouli |Friday   |3        |
|Kalyan Krishna |Friday   |3        |
|Sabbir Khan    |Friday   |3        |
|H. Vinoth      |Friday   |3        |
|Remo D'Souza   |Friday   |3        |
|Kabir Khan     |Friday   |3        |
|Surender Reddy |Friday   |3        |
|Farhad Samji   |Friday   |3        |
|K.V. Anand     |Friday   |3        |
|Boyapati Srinu |Friday   |3        |
|Venky Atluri   |Friday   |3        |
+---------------+---------+---------+
only showing top 20 rows



In [103]:
#58) Write a query to get OTT platofrm and director wise films released?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .groupBy("OTT_Platform","Director")\
     .agg(count("FilmID").alias("FilmCount"))\
     .orderBy(desc("FilmCount")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 105, Finished, Available, Finished)

+------------------+------------------+---------+
|OTT_Platform      |Director          |FilmCount|
+------------------+------------------+---------+
|Amazon Prime Video|Anil Ravipudi     |5        |
|Amazon Prime Video|Trivikram Srinivas|5        |
|Amazon Prime Video|Prashanth Neel    |4        |
|Amazon Prime Video|K.S. Ravindra     |4        |
|Amazon Prime Video|Vamshi Paidipally |4        |
|Netflix           |Rohit Shetty      |4        |
|Amazon Prime Video|Vikram K. Kumar   |4        |
|Amazon Prime Video|Sukumar           |4        |
|Amazon Prime Video|A.R. Murugadoss   |4        |
|Amazon Prime Video|Siva              |4        |
|Amazon Prime Video|Sundar C.         |4        |
|Amazon Prime Video|R. Ajay Gnanamuthu|3        |
|Amazon Prime Video|K.V. Anand        |3        |
|Amazon Prime Video|Lokesh Kanagaraj  |3        |
|Amazon Prime Video|Surender Reddy    |3        |
|Netflix           |S.S. Rajamouli    |3        |
|Amazon Prime Video|Anand Shankar     |3        |


In [104]:
#59)Write a query to get director wise films released on Friday only?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .filter(col("WeekName") == "Friday")\
     .groupBy("Director").agg(count("FilmID").alias("FilmCount"))\
     .orderBy(desc("FilmCount")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 106, Finished, Available, Finished)

+--------------+---------+
|Director      |FilmCount|
+--------------+---------+
|Rohit Shetty  |6        |
|Mohit Suri    |4        |
|Siva          |4        |
|Remo D'Souza  |3        |
|Sundar C.     |3        |
|Kabir Khan    |3        |
|Farhad Samji  |3        |
|P.S. Mithran  |3        |
|S.S. Rajamouli|3        |
|Meghna Gulzar |3        |
|Kalyan Krishna|3        |
|Anil Ravipudi |3        |
|Prashanth Neel|3        |
|Venky Atluri  |3        |
|Surender Reddy|3        |
|H. Vinoth     |3        |
|Boyapati Srinu|3        |
|Anees Bazmee  |3        |
|Sukumar       |3        |
|Sabbir Khan   |3        |
+--------------+---------+
only showing top 20 rows



In [105]:
#60)Write a query to get films based on IMDb reating between 6.5 and 7.7?
#df_bo.select("Title","IMDb_Rating").filter((col("IMDb_Rating") >= 6.5) & (col("IMDB_Rating") <= 7.7)).show() 
df_bo.select("Title","IMDb_Rating").where("IMDb_Rating BETWEEN 6.5 AND 7.7").show() #Optimized way

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 107, Finished, Available, Finished)

+--------------------+-----------+
|               Title|IMDb_Rating|
+--------------------+-----------+
|               Sanju|        7.6|
|      Janatha Garage|        7.2|
|            Maanikya|        6.5|
|         Srimanthudu|        7.5|
|   Kadaikutty Singam|        6.8|
|       Irumbu Thirai|        7.6|
|Abrahaminte Santh...|        6.7|
|              Veeram|        6.5|
|Tanu Weds Manu Re...|        7.6|
|     Geetha Govindam|        7.7|
|    Imaikkaa Nodigal|        7.3|
|      Bheeshmaparvam|        7.6|
|          Ek Villain|        6.6|
|      Njan Prakashan|        7.7|
|         Race Gurram|        7.3|
|              Rx 100|        6.9|
|           Padmaavat|        7.1|
|              Sultan|        7.1|
|            Varathan|        7.5|
|               Stree|        7.5|
+--------------------+-----------+
only showing top 20 rows



In [106]:
#61)Write a query to get director,films and IMDb ratings?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .select("Director","Title","IMDb_Rating").show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 108, Finished, Available, Finished)

+-----------------------------+------------------------+-----------+
|Director                     |Title                   |IMDb_Rating|
+-----------------------------+------------------------+-----------+
|Rajkumar Hirani              |Sanju                   |7.6        |
|Rohit Shetty                 |Simmba                  |5.5        |
|Koratala Siva                |Janatha Garage          |7.2        |
|A.R. Murugadoss              |Kaththi                 |8.1        |
|Sudeep, Nanda Kishore, Mahesh|Maanikya                |6.5        |
|Koratala Siva                |Srimanthudu             |7.5        |
|Pandiraj                     |Kadaikutty Singam       |6.8        |
|P.S. Mithran                 |Irumbu Thirai           |7.6        |
|Shaji Padoor                 |Abrahaminte Santhathikal|6.7        |
|Siva                         |Veeram                  |6.5        |
|Aanand L. Rai                |Tanu Weds Manu Returns  |7.6        |
|Parasuram                    |Gee

In [107]:
#62) Write a query to get films with highest budget based on flop verdict?
df_bo.select("Title","Budget_in_Crores","Verdict").where("Verdict == 'Flop'").orderBy(desc("Budget_in_Crores")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 109, Finished, Available, Finished)

+---------------------+----------------+-------+
|Title                |Budget_in_Crores|Verdict|
+---------------------+----------------+-------+
|Thugs of Hindostan   |275.0           |Flop   |
|83                   |200.0           |Flop   |
|Annaatthe            |180.0           |Flop   |
|Valimai              |150.0           |Flop   |
|Race 3               |150.0           |Flop   |
|Vikram Vedha         |150.0           |Flop   |
|Vivegam              |120.0           |Flop   |
|Spyder               |120.0           |Flop   |
|Street Dancer 3D     |100.0           |Flop   |
|Godfather            |100.0           |Flop   |
|Bro                  |100.0           |Flop   |
|Baaghi 3             |100.0           |Flop   |
|Tubelight            |100.0           |Flop   |
|Chandu Champion      |90.0            |Flop   |
|Thangalaan           |80.0            |Flop   |
|Action Jackson       |80.0            |Flop   |
|Sardaar Gabbar Singh |75.0            |Flop   |
|Ayalaan            

In [108]:
#63)Write a query to get total number of directors?
dircount = df_dir.select("Director").count()
print(dircount)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 110, Finished, Available, Finished)

384


In [109]:
#64)Write a query to get vedridct wise total films released?
df_bo.groupBy("Verdict").agg(count("FilmID").alias("FilmCount")).orderBy(desc("FilmCount")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 111, Finished, Available, Finished)

+--------------------+---------+
|Verdict             |FilmCount|
+--------------------+---------+
|Blockbuster         |166      |
|Hit                 |96       |
|Super Hit           |89       |
|Disaster            |68       |
|Flop                |57       |
|Below Average       |47       |
|Average             |42       |
|Above Average       |23       |
|All Time Blockbuster|16       |
+--------------------+---------+



In [110]:
#65)Write a query to get top 10 directors based number of films?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .groupBy("Director").agg(count("FilmID").alias("FilmCount"))\
     .orderBy(desc("FilmCount")).limit(10).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 112, Finished, Available, Finished)

+------------------+---------+
|Director          |FilmCount|
+------------------+---------+
|Trivikram Srinivas|6        |
|Rohit Shetty      |6        |
|Anil Ravipudi     |5        |
|Koratala Siva     |5        |
|A.R. Murugadoss   |5        |
|Boyapati Srinu    |5        |
|Siva              |5        |
|Maruthi Dasari    |4        |
|Lokesh Kanagaraj  |4        |
|Siddharth Anand   |4        |
+------------------+---------+



In [111]:
#66)Write a query to get top 5 directors based on world wide collections and industry wise?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .groupBy("Industry","Director").agg(sum("Worldwide_Collection_in_Crores").alias("WW_Collections"))\
     .withColumn("Rank",row_number().over(Window.partitionBy("Industry").orderBy(desc("WW_Collections"))))\
     .filter(col("Rank") <= 5)\
     .select("Industry","Director","WW_Collections").show(25,truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 113, Finished, Available, Finished)

+----------+---------------------------------------+-----------------+
|Industry  |Director                               |WW_Collections   |
+----------+---------------------------------------+-----------------+
|Bollywood |Siddharth Anand                        |2229.45          |
|Bollywood |Nitesh Tiwari                          |2122.3           |
|Bollywood |Rajkumar Hirani                        |1834.5           |
|Bollywood |Rohit Shetty                           |1656.6           |
|Bollywood |Sandeep Reddy Vanga                    |1292.0           |
|Kollywood |Lokesh Kanagaraj                       |1350.13          |
|Kollywood |S. Shankar                             |1089.83          |
|Kollywood |Nelson Dilipkumar                      |952.88           |
|Kollywood |Mani Ratnam, Sruti Harihara Subramanian|832.99           |
|Kollywood |Siva                                   |727.9000000000001|
|Mollywood |Vysakh                                 |300.65           |
|Molly

In [112]:
#67)Write a query to get top 3 directors based on India gross collections in Bollowood industry?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .filter(col("Industry") == "Bollywood")\
     .groupBy("Director").agg(sum("India_Gross_Collection_in_Crores").alias("Ind_Gross_Collections"))\
     .sort(desc("Ind_Gross_Collections")).limit(3).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 114, Finished, Available, Finished)

+---------------+---------------------+
|       Director|Ind_Gross_Collections|
+---------------+---------------------+
|Siddharth Anand|   1560.9099999999999|
|   Rohit Shetty|               1246.2|
|Rajkumar Hirani|               1199.5|
+---------------+---------------------+



In [113]:
#68) Write a query to get top 3 directors based on India gross collections in Tollowood industry?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .filter(col("Industry") == "Tollywood")\
     .groupBy("Director").agg(sum("India_Gross_Collection_in_Crores").alias("Ind_Gross_Collections"))\
     .orderBy(desc("Ind_Gross_Collections")).limit(3).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 115, Finished, Available, Finished)

+------------------+---------------------+
|          Director|Ind_Gross_Collections|
+------------------+---------------------+
|    S.S. Rajamouli|              2848.85|
|        Nag Ashwin|               840.25|
|Trivikram Srinivas|               711.65|
+------------------+---------------------+



In [114]:
#69) Write a query to get top 3 directors based on India gross collections in Kollowood industry?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .filter(col("Industry") == "Kollywood")\
     .groupBy("Director").agg(sum("India_Gross_Collection_in_Crores").alias("Ind_Gross_Collections"))\
     .orderBy(desc("Ind_Gross_Collections")).limit(3).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 116, Finished, Available, Finished)

+-----------------+---------------------+
|         Director|Ind_Gross_Collections|
+-----------------+---------------------+
| Lokesh Kanagaraj|    956.8799999999999|
|       S. Shankar|               826.83|
|Nelson Dilipkumar|               669.88|
+-----------------+---------------------+



In [115]:
#70)Write a query to get top 3 directors based on India gross collections in Mollowood industry?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .filter(col("Industry") == "Mollywood")\
     .groupBy("Director").agg(sum("India_Gross_Collection_in_Crores").alias("Ind_Gross_Collections"))\
     .orderBy(desc("Ind_Gross_Collections")).limit(3).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 117, Finished, Available, Finished)

+--------------+---------------------+
|      Director|Ind_Gross_Collections|
+--------------+---------------------+
|        Vysakh|               208.55|
|   Chidambaram|               167.65|
|Jithu Madhavan|               144.49|
+--------------+---------------------+



In [116]:
#71)Write a query to get top 3 directors based on India gross collections in Sandalwood industry?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .filter(col("Industry") == "Sandalwood")\
     .groupBy("Director").agg(sum("India_Gross_Collection_in_Crores").alias("Ind_Gross_Colelctions"))\
     .sort(desc("Ind_Gross_Colelctions")).limit(3).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 118, Finished, Available, Finished)

+------------------+---------------------+
|          Director|Ind_Gross_Colelctions|
+------------------+---------------------+
|    Prashanth Neel|              1259.85|
|     Rishab Shetty|               363.82|
|Santhosh Ananddram|   164.39999999999998|
+------------------+---------------------+



In [117]:
#72)Write to get total number of generes?
gencount = df_gen.select("GenreID").count()
print(gencount)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 119, Finished, Available, Finished)

159


In [118]:
#73) Write query to get director,language,genere films count?
df_bo.join(df_dir,df_bo["DirectorID"] == df_dir["Director_ID"],"inner")\
     .join(df_lan,df_bo["LanguageID"] == df_lan["LanguageID"],"inner")\
     .join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .groupBy("Language","Director","Genre")\
     .agg(count("FilmID").alias("FilmCount")).show(truncate=False)


StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 120, Finished, Available, Finished)

+---------+-------------------------+-------------------------------------------+---------+
|Language |Director                 |Genre                                      |FilmCount|
+---------+-------------------------+-------------------------------------------+---------+
|Hindi    |Anurag Basu, Shubzz      |Action, Adventure, Comedy, Musical, Mystery|1        |
|Hindi    |Imtiaz Ali               |Drama, Romance                             |1        |
|Hindi    |Mahesh Manjrekar         |Action, Crime, Drama                       |1        |
|Tamil    |Vetrimaaran              |Action, Drama                              |1        |
|Telugu   |Rahul Sankrityan         |Comedy, Horror, Thriller                   |1        |
|Tamil    |Atlee                    |Action, Drama, Sport                       |1        |
|Kannada  |Tarun Sudhir             |Action                                     |1        |
|Malayalam|Jibi Joju                |Comedy, Drama                              

In [119]:
#74)Write a query to genere wise budget?
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .groupBy("Genre").agg(sum("Budget_in_Crores").alias("TTL_Budget")).sort(desc("TTL_Budget")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 121, Finished, Available, Finished)

+--------------------+----------+
|               Genre|TTL_Budget|
+--------------------+----------+
|       Action, Drama|    3113.0|
|Action, Crime, Dr...|    2937.0|
|Action, Crime, Th...|    1560.0|
|Action, Drama, Th...|    1457.5|
|Comedy, Drama, Ro...|    1308.5|
|Action, Adventure...|    1299.0|
|    Action, Thriller|    1277.0|
|Action, Adventure...|    1100.0|
|Action, Comedy, D...|     915.0|
|Action, Crime, Drama|     880.0|
|      Drama, Romance|     720.0|
|       Comedy, Drama|     715.0|
|Action, Sci-Fi, T...|     655.0|
|Action, Comedy, D...|     600.0|
|Action, Adventure...|     530.0|
|Action, Adventure...|     500.0|
|Action, Adventure...|     500.0|
|      Action, Comedy|     485.0|
|Action, Comedy, C...|     480.0|
|     Comedy, Romance|     457.0|
+--------------------+----------+
only showing top 20 rows



In [120]:
#75) Write a query to get genere wise first day worldwide collections?
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .groupBy("Genre").agg(sum("First_Day_Collection_Worldwide_in_Crores").alias("1st_Day_WW_Collections"))\
     .orderBy(desc("1st_Day_WW_Collections")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 122, Finished, Available, Finished)

+---------------------------------------------------+----------------------+
|Genre                                              |1st_Day_WW_Collections|
+---------------------------------------------------+----------------------+
|Action, Drama                                      |1433.05               |
|Action, Crime, Drama, Thriller                     |1396.49               |
|Action, Drama, Thriller                            |532.8600000000001     |
|Action, Crime, Thriller                            |452.2                 |
|Action, Adventure, Drama                           |425.31999999999994    |
|Action, Comedy, Drama                              |391.04999999999995    |
|Action, Adventure, Thriller                        |383.5                 |
|Action, Crime, Drama                               |345.79999999999995    |
|Action, Thriller                                   |326.45                |
|Comedy, Drama, Romance                             |294.91999999999996    |

In [121]:
#76)Write a query to get genere wise overseas collections?
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .groupBy("Genre").agg(sum("Overseas_Collection_in_Crores").alias("Overses_Collections"))\
     .sort(desc("Overses_Collections")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 123, Finished, Available, Finished)

+---------------------------------------------------+-------------------+
|Genre                                              |Overses_Collections|
+---------------------------------------------------+-------------------+
|Action, Crime, Drama, Thriller                     |2043.1300000000003 |
|Action, Drama                                      |1598.18            |
|Action, Biography, Drama, Sport                    |1535.3             |
|Comedy, Drama                                      |872.9599999999999  |
|Drama, Music                                       |846.4              |
|Action, Adventure, Thriller                        |791.74             |
|Comedy, Drama, Romance                             |723.85             |
|Action, Adventure, Drama                           |700.46             |
|Action, Drama, Thriller                            |667.55             |
|Action, Crime, Thriller                            |541.9              |
|Action, Adventure, Comedy, Drama     

In [122]:
#77) Write a query to get genere wise India gross collections?
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .groupBy("Genre").agg(sum("India_Gross_Collection_in_Crores").alias("Ind_Gross_Collections"))\
     .sort(desc("Ind_Gross_Collections")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 124, Finished, Available, Finished)

+---------------------------------------------------+---------------------+
|Genre                                              |Ind_Gross_Collections|
+---------------------------------------------------+---------------------+
|Action, Crime, Drama, Thriller                     |6212.3200000000015   |
|Action, Drama                                      |5851.500000000001    |
|Comedy, Drama, Romance                             |2334.3               |
|Action, Adventure, Thriller                        |2079.98              |
|Action, Drama, Thriller                            |2044.3500000000001   |
|Action, Crime, Thriller                            |1957.7600000000002   |
|Action, Adventure, Drama                           |1665.8000000000002   |
|Action, Comedy, Drama                              |1498.85              |
|Action, Thriller                                   |1494.4               |
|Action, Crime, Drama                               |1466.8               |
|Comedy, Hor

In [123]:
#78)Write a query to get genere wise top 2 longest run time movies?
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .withColumn("Rank",row_number().over(Window.partitionBy("Genre").orderBy(desc("Runtime_(mins)"))))\
     .filter(col("Rank") <= 2)\
     .select("Genre","Title","Runtime_(mins)").show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 125, Finished, Available, Finished)

+-----------------------------------------------------+-----------------------+--------------+
|Genre                                                |Title                  |Runtime_(mins)|
+-----------------------------------------------------+-----------------------+--------------+
|Action                                               |Kaatera                |183           |
|Action                                               |Yajamana               |164           |
|Action, Adventure                                    |Thugs of Hindostan     |164           |
|Action, Adventure                                    |RDX: Robert Dony Xavier|151           |
|Action, Adventure, Comedy                            |Total Dhamaal          |130           |
|Action, Adventure, Comedy                            |Dishoom                |124           |
|Action, Adventure, Comedy, Drama                     |Bajrangi Bhaijaan      |163           |
|Action, Adventure, Comedy, Drama                 

In [124]:
#79)Write a query to get genere wise bottom 2 shortest runtime movies?
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .withColumn("Rank",row_number().over(Window.partitionBy("Genre").orderBy(asc("Runtime_(mins)"))))\
     .filter(col("Rank") <=2)\
     .select("Genre","Title","Runtime_(mins)").show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 126, Finished, Available, Finished)

+-----------------------------------------------------+-----------------------+--------------+
|Genre                                                |Title                  |Runtime_(mins)|
+-----------------------------------------------------+-----------------------+--------------+
|Action                                               |Garudan                |133           |
|Action                                               |Motta Shiva Ketta Shiva|151           |
|Action, Adventure                                    |RDX: Robert Dony Xavier|151           |
|Action, Adventure                                    |Thugs of Hindostan     |164           |
|Action, Adventure, Comedy                            |Dishoom                |124           |
|Action, Adventure, Comedy                            |Total Dhamaal          |130           |
|Action, Adventure, Comedy, Drama                     |A.R.M                  |150           |
|Action, Adventure, Comedy, Drama                 

In [125]:
#80) Write a query to get verdict, genere wise films released?
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .groupBy("Verdict","Genre")\
     .agg(count("FilmID").alias("FilmCount")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 127, Finished, Available, Finished)

+-------------+------------------------------+---------+
|Verdict      |Genre                         |FilmCount|
+-------------+------------------------------+---------+
|Hit          |Action                        |2        |
|Disaster     |Comedy, Drama, Romance        |3        |
|Blockbuster  |Drama, Family, Romance        |1        |
|Above Average|Action, Romance, Thriller     |1        |
|Hit          |Drama, Family, Romance        |1        |
|Disaster     |Comedy, Drama                 |3        |
|Disaster     |Action, Adventure, Fantasy    |1        |
|Below Average|Action, Comedy                |2        |
|Blockbuster  |Biography, Drama              |2        |
|Hit          |Action, Comedy, Drama, Romance|2        |
|Hit          |Crime, Thriller               |1        |
|Flop         |Comedy, Drama                 |1        |
|Blockbuster  |Action, Drama, Romance        |4        |
|Blockbuster  |Action, Drama, History, War   |1        |
|Super Hit    |Action, Adventur

In [126]:
#81)Write a query to get genere, OTT platform wise films count?
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .groupBy("OTT_Platform","Genre").agg(count("FilmID").alias("FilmCount")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 128, Finished, Available, Finished)

+------------------+---------------------------------------------+---------+
|OTT_Platform      |Genre                                        |FilmCount|
+------------------+---------------------------------------------+---------+
|Amazon Prime Video|Comedy, Family, Romance                      |2        |
|Netflix           |Comedy, Romance                              |5        |
|ZEE5              |Action, Drama, History, War                  |1        |
|Netflix           |Action, Crime, Romance, Thriller             |1        |
|Amazon Prime Video|Biography, Drama, Romance                    |1        |
|Disney+ Hotstar   |Comedy, Drama                                |1        |
|ZEE5              |Action, Adventure, Drama, Thriller           |1        |
|Amazon Prime Video|Drama, Fantasy                               |1        |
|Netflix           |Action, Crime, Drama                         |2        |
|SonyLIV           |Action, Crime, Thriller                      |1        |

In [127]:
#82)Write a query to get genere wise films count?
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .groupBy("Genre").agg(count("FilmID").alias("FilmCount")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 129, Finished, Available, Finished)

+----------------------------------------+---------+
|Genre                                   |FilmCount|
+----------------------------------------+---------+
|Action, Thriller                        |19       |
|Action, Drama, Music, Romance           |1        |
|Crime, Thriller                         |1        |
|Action, Adventure, Fantasy              |3        |
|Action, Adventure, Drama, Fantasy       |1        |
|Action, Comedy, Crime, Romance          |2        |
|Action, Drama, Romance, Sci-Fi, Thriller|1        |
|Drama, Musical, Romance                 |1        |
|Action, Crime, Drama, History           |1        |
|Comedy, Drama                           |22       |
|Romance                                 |4        |
|Biography, Drama, Sport                 |3        |
|Comedy, Drama, Romance                  |29       |
|Action, Drama                           |44       |
|Comedy, Family, Romance                 |2        |
|Action, Adventure, Comedy, Drama        |2   

In [128]:
#83)Write a query to get genere wise films count  in Tollywood Industry? # Filtering the records using where command
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .where("Industry == 'Tollywood'")\
     .groupBy("Genre").agg(count("FilmID").alias("FilmCount")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 130, Finished, Available, Finished)

+---------------------------------+---------+
|Genre                            |FilmCount|
+---------------------------------+---------+
|Action, Thriller                 |7        |
|Action, Adventure, Fantasy       |1        |
|Action, Adventure, Drama, Fantasy|1        |
|Action, Comedy, Crime, Romance   |1        |
|Action, Crime, Drama, History    |1        |
|Comedy, Drama                    |2        |
|Romance                          |4        |
|Comedy, Drama, Romance           |8        |
|Action, Drama                    |19       |
|Comedy, Family, Romance          |1        |
|Comedy, Horror, Thriller         |2        |
|Action, Drama, Romance           |5        |
|Action, Drama, Thriller          |8        |
|Crime, Horror, Mystery, Thriller |1        |
|Biography, Drama                 |2        |
|Comedy, Drama, Fantasy           |2        |
|Comedy, Romance                  |4        |
|Drama, Romance                   |6        |
|Action, Drama, Romance, Sport    

In [129]:
#84)Write a query to get genere wise films count in Kollywood Industry?
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .where("Industry == 'Kollywood'")\
     .groupBy("Genre").agg(count("FilmID").alias("FilmCount")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 131, Finished, Available, Finished)

+----------------------------------------+---------+
|Genre                                   |FilmCount|
+----------------------------------------+---------+
|Action, Thriller                        |7        |
|Action, Adventure, Fantasy              |1        |
|Action, Drama, Romance, Sci-Fi, Thriller|1        |
|Comedy, Drama                           |3        |
|Comedy, Drama, Romance                  |3        |
|Action, Drama                           |17       |
|Action, Comedy, Drama, Fantasy, Thriller|1        |
|Action, Drama, Romance                  |3        |
|Action, Drama, Thriller                 |8        |
|Biography, Drama                        |1        |
|Comedy, Romance                         |1        |
|Drama, Romance                          |1        |
|Comedy, Drama, Horror                   |1        |
|Action, Horror, Thriller                |1        |
|Drama                                   |2        |
|Action, Comedy                          |2   

In [130]:
#85)Write a query to get genere wise films count in Mollywood Industry?
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .where("Industry == 'Mollywood'")\
     .groupBy("Genre").agg(count("FilmID").alias("FilmCount")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 132, Finished, Available, Finished)

+---------------------------------------+---------+
|Genre                                  |FilmCount|
+---------------------------------------+---------+
|Action, Thriller                       |2        |
|Crime, Thriller                        |1        |
|Drama, Musical, Romance                |1        |
|Comedy, Drama                          |5        |
|Comedy, Drama, Romance                 |6        |
|Action, Drama                          |1        |
|Comedy, Family, Romance                |1        |
|Action, Adventure, Comedy, Drama       |1        |
|Action, Drama, Thriller                |6        |
|Action, Drama, History, War            |2        |
|Comedy, Romance                        |4        |
|Drama                                  |2        |
|Drama, Horror, Thriller                |1        |
|Horror, Mystery, Thriller              |2        |
|Action, Comedy                         |2        |
|Action, Adventure                      |1        |
|Comedy, Dra

In [131]:
#86)Write a query to get genere wise films count in Bollywood Industry?
df_bo.join(df_gen,df_bo["GenreId"] == df_gen["GenreId"],"inner")\
     .where("Industry == 'Bollywood'")\
     .groupBy("Genre").agg(count("FilmID").alias("FilmCount")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 133, Finished, Available, Finished)

+--------------------------------+---------+
|Genre                           |FilmCount|
+--------------------------------+---------+
|Action, Thriller                |1        |
|Action, Drama, Music, Romance   |1        |
|Action, Adventure, Fantasy      |1        |
|Action, Comedy, Crime, Romance  |1        |
|Comedy, Drama                   |12       |
|Biography, Drama, Sport         |3        |
|Comedy, Drama, Romance          |12       |
|Action, Drama                   |2        |
|Action, Adventure, Comedy, Drama|1        |
|Drama, History                  |1        |
|Drama, Music                    |2        |
|Comedy, Horror, Thriller        |1        |
|Biography, Drama, War           |1        |
|Mystery, Thriller               |1        |
|Action, Drama, Romance          |1        |
|Action, Drama, Thriller         |5        |
|Documentary, Sport              |1        |
|Action, Drama, History, War     |2        |
|Drama, History, Sport           |1        |
|Biography

In [132]:
#87)Write a query to get genere wise films count in Sandalwood Industry?
df_bo.join(df_gen,df_bo["GenreID"] == df_gen["GenreID"],"inner")\
     .where("Industry == 'Sandalwood'")\
     .groupBy("Genre").agg(count("FilmID").alias("FilmCount")).show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 134, Finished, Available, Finished)

+-----------------------------------------+---------+
|Genre                                    |FilmCount|
+-----------------------------------------+---------+
|Action, Thriller                         |2        |
|Action, Drama                            |5        |
|Action, Drama, Romance                   |2        |
|Action, Drama, Thriller                  |1        |
|Drama, Romance                           |1        |
|Action, Crime, Drama, Mystery, Thriller  |1        |
|Action, Adventure, Comedy, Drama, Fantasy|1        |
|Action, Comedy, Musical                  |1        |
|Action, Romance                          |1        |
|Action, Adventure, Drama, Thriller       |1        |
|Action, Crime, Drama, Thriller           |3        |
|Adventure, Comedy, Drama                 |1        |
|Action, Comedy, Drama                    |1        |
|Action, Crime, Thriller                  |2        |
|Action, Crime                            |3        |
|Action, Drama, Sport       

In [133]:
#88)Write a query to get lead actors/actress wise,genere and films count?
df_bo.groupBy("Lead_Actor/Actress")\
     .agg(count_distinct("GenreID").alias("GenreCount"),count("FilmID").alias("FilmCount")).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 135, Finished, Available, Finished)

+--------------------+----------+---------+
|  Lead_Actor/Actress|GenreCount|FilmCount|
+--------------------+----------+---------+
|      Anushka Sharma|         1|        1|
|   Naveen Polishetty|         2|        2|
|    Thennal Abhilash|         1|        1|
|         Vishwak Sen|         3|        3|
|     Divyendu Sharma|         1|        1|
|     Pankaj Tripathi|         1|        1|
|    Rajsekhar Aningi|         1|        1|
|      Kareena Kapoor|         1|        1|
|             Sathish|         1|        1|
|        Naga Shaurya|         1|        1|
|       Suniel Shetty|         1|        1|
|        Akshay Kumar|        20|       21|
|        Joseph Vijay|         8|       12|
|Sushant Singh Rajput|         1|        1|
|         Adah Sharma|         1|        1|
|   Anand Deverakonda|         1|        1|
|       Saif Ali Khan|         2|        2|
|   Sidharth Malhotra|         3|        3|
|               Jiiva|         2|        2|
|         Zaira Wasim|         1

In [134]:
#89)Write a query to get 5th rank movie based on Worldwide total collections?
df_bo.withColumn("Rank",row_number().over(Window.orderBy(desc("Worldwide_Collection_in_Crores"))))\
     .filter(col("Rank") == 5).select("Title","Worldwide_Collection_in_Crores").show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 136, Finished, Available, Finished)

+-----+------------------------------+
|Title|Worldwide_Collection_in_Crores|
+-----+------------------------------+
|Jawan|                        1160.0|
+-----+------------------------------+



In [135]:
#90)Write a query to get 5th rank movie by industry wise based on First day & worldwide collections?
df_bo.withColumn("Rank",row_number().over(Window.partitionBy("Industry").orderBy(desc("First_Day_Collection_Worldwide_in_Crores"))))\
     .filter(col("Rank") == 5)\
     .select("Industry","Title","First_Day_Collection_Worldwide_in_Crores").show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 137, Finished, Available, Finished)

+----------+--------------------------+----------------------------------------+
|Industry  |Title                     |First_Day_Collection_Worldwide_in_Crores|
+----------+--------------------------+----------------------------------------+
|Bollywood |Stree 2: Sarkate Ka Aatank|80.0                                    |
|Kollywood |Jailer                    |96.6                                    |
|Mollywood |Turbo                     |15.7                                    |
|Sandalwood|Roberrt                   |16.9                                    |
|Tollywood |Devara: Part 1            |142.0                                   |
+----------+--------------------------+----------------------------------------+



In [136]:
#91)Write a query to get 3rd rank movie by industry wise based on IMDb Ratings?
df_bo.withColumn("Rank",row_number().over(Window.partitionBy("Industry").orderBy(desc("IMDb_Rating"))))\
     .filter(col("Rank") == 3)\
     .select("Industry","Title","IMDb_Rating").show(truncate=False)

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 138, Finished, Available, Finished)

+----------+----------------+-----------+
|Industry  |Title           |IMDb_Rating|
+----------+----------------+-----------+
|Bollywood |Sachin          |8.5        |
|Kollywood |Maharaja        |8.5        |
|Mollywood |Bangalore Days  |8.3        |
|Sandalwood|K.G.F: Chapter 2|8.2        |
|Tollywood |Mahanati        |8.4        |
+----------+----------------+-----------+



In [141]:
#92)Calculate YoY% Budget growth?
df_bo.withColumn("Year",year("Release_Date"))\
     .groupBy("Year")\
     .agg(sum("Budget_in_Crores").alias("Total_Budget"))\
     .withColumn("Previous_Yr_Bduget",lag("Total_Budget").over(Window.orderBy("Year")))\
     .withColumn("YoY_Budget_Growth",((col("Total_Budget") - col("Previous_Yr_Bduget")) / col("Previous_Yr_Bduget"))*100).show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 143, Finished, Available, Finished)

+----+------------+------------------+-------------------+
|Year|Total_Budget|Previous_Yr_Bduget|  YoY_Budget_Growth|
+----+------------+------------------+-------------------+
|2014|      1820.0|              NULL|               NULL|
|2015|      1906.0|            1820.0|  4.725274725274725|
|2016|      2142.0|            1906.0| 12.381951731374606|
|2017|      3291.0|            2142.0| 53.641456582633054|
|2018|      3679.0|            3291.0| 11.789729565481617|
|2019|      2544.0|            3679.0|-30.850774667029086|
|2020|      1387.0|            2544.0|-45.479559748427675|
|2021|      1855.0|            1387.0| 33.741888968997834|
|2022|      5722.0|            1855.0|  208.4636118598383|
|2023|      5877.0|            5722.0| 2.7088430618664803|
|2024|      4430.0|            5877.0|-24.621405478985878|
+----+------------+------------------+-------------------+



In [145]:
#93)  Calculate YoY% Worldwide total collelctions growth?
df_bo.withColumn("Year",year("Release_Date"))\
     .groupBy("Year")\
     .agg(sum("Worldwide_Collection_in_Crores").alias("WW_Collections"))\
     .withColumn("Previous_Yr_WW_Collections",lag("WW_Collections").over(Window.orderBy("Year")))\
     .withColumn("YoY_WW_Collections_Growth",((col("WW_Collections") - col("Previous_Yr_WW_Collections")) / col("Previous_Yr_WW_Collections"))*100)\
     .show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 147, Finished, Available, Finished)

+----+------------------+--------------------------+-------------------------+
|Year|    WW_Collections|Previous_Yr_WW_Collections|YoY_WW_Collections_Growth|
+----+------------------+--------------------------+-------------------------+
|2014| 4645.000000000001|                      NULL|                     NULL|
|2015| 5470.200000000001|         4645.000000000001|       17.765339074273403|
|2016| 7088.799999999999|         5470.200000000001|       29.589411721692045|
|2017|            9659.2|         7088.799999999999|        36.26001579957118|
|2018| 8275.300000000001|                    9659.2|      -14.327273480205397|
|2019|           8242.72|         8275.300000000001|     -0.39370173890978866|
|2020|           2348.75|                   8242.72|       -71.50515849137177|
|2021|2711.6499999999996|                   2348.75|       15.450771687067574|
|2022| 9853.160000000002|        2711.6499999999996|        263.3640034665242|
|2023|13461.720000000001|         9853.160000000002|

In [147]:
#94)Calculate YoY% Indian Gross colelctions growth?
df_bo.withColumn("Year",year("Release_Date"))\
     .groupBy("Year").agg(sum("India_Gross_Collection_in_Crores").alias("Ind_Gross_Colelctions"))\
     .withColumn("Previous_Yr_Collections",lag("Ind_Gross_Colelctions").over(Window.orderBy("Year")))\
     .withColumn("YoY_Ind_Gross_Colelctions_Growth",((col("Ind_Gross_Colelctions")-col("Previous_Yr_Collections")) / col("Previous_Yr_Collections")) * 100)\
     .show()

StatementMeta(, ac46d685-f297-4931-ab8e-38c7a1199d6c, 149, Finished, Available, Finished)

+----+---------------------+-----------------------+--------------------------------+
|Year|Ind_Gross_Colelctions|Previous_Yr_Collections|YoY_Ind_Gross_Colelctions_Growth|
+----+---------------------+-----------------------+--------------------------------+
|2014|    3711.099999999999|                   NULL|                            NULL|
|2015|   3935.2999999999997|      3711.099999999999|                6.04133545310018|
|2016|    4459.900000000001|     3935.2999999999997|              13.330622824181152|
|2017|   6852.9000000000015|      4459.900000000001|               53.65591156752395|
|2018|    6327.400000000001|     6852.9000000000015|              -7.668286418888365|
|2019|    6655.580000000001|      6327.400000000001|               5.186648544425835|
|2020|              1966.35|      6655.580000000001|              -70.45561769222218|
|2021|               2238.4|                1966.35|              13.835278561802333|
|2022|    7716.210000000003|                 2238.4|  

# Thank you all